# Project 3 - Risk Factors for Attrition in Longitudinal Studies

In [ ]:
## Mounts runtime to google drive, allowing access to cloud files
## Also loads rpy2 kernel, allowing for R code chunks
from google.colab import drive
drive.mount('/content/gdrive')
%reload_ext rpy2.ipython

In [ ]:
!pip install lifelines -q
!pip install tableone -q

In [ ]:
# Set-Up
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
import os
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

datafolder = '/content/gdrive/My Drive/Attrition'

In [ ]:
%%R
##Loads libraries required for analysis
pack <- "/content/gdrive/My Drive/R/packages"

library(data.table, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library(dplyr, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library("ggpubr", lib.loc = pack)
library("ggplot2", lib.loc = pack)
library("ggpubr", lib.loc = pack)
library("survival", lib.loc = pack)
library("survminer", lib.loc = pack)
library(tidyr, lib.loc = pack)
library(knitr)

In [ ]:
def PrepKM(data, cohort_dict):
  new_data = pd.DataFrame()

  flag = 0
  data.loc[:,"ID"] = data.iloc[:,0]
  data = data.loc[:,data.columns.isin(["ID"] + list(cohort_dict.values()))]
  data = data.groupby("ID").fillna(method="bfill")
  for key, val in cohort_dict.items():
    if "IS_" in key:
      new_data.loc[:,key] = [np.nan if pd.isnull(i) else 1 if i >= int(val.split("/")[1]) else 0 for i in data[val.split("/")[0]]]
    elif val not in data.columns:
      print(f"Error: {val} not found")
    else:
      new_data.loc[:,key] = data[val]
      if flag == 1:
        new_data.loc[~new_data.ID.duplicated(keep="first"),f"BL_{key}"] = new_data.loc[:,key]
        new_data.loc[:,f"BL_{key}"] = new_data.loc[:,f"BL_{key}"].fillna(method = "ffill")
      if key == "BL_AGE":
        flag = 1

  if "daysB" not in new_data.columns:
   new_data.insert(loc = 1, column = "daysB", value = new_data.DATE - new_data.BL_DATE)
  new_data.loc[:,"BL_durdx"] = (new_data.BL_AGE - new_data.DX_AGE).round(2)
  new_data.loc[:,"durdx"] = (new_data.BL_durdx + (new_data.daysB / 365.25)).round(2)
  new_data.loc[:,"COG"] = new_data.groupby("ID").COG.fillna(method = "ffill")

  TD_data = new_data.copy()
  TD_data.loc[:,"LTF_TD"] = 0 #[0 if k == 0 else 1 if i > j else 0 for i, j, k in zip(TD_data.DATE, TD_data.CENS_DT, TD_data.LTF)]
  TD_data.loc[(~TD_data.duplicated(subset = "ID",keep="last")) & (TD_data.LTF == 1), "LTF_TD"] = 1

  TD_data.loc[:,"TSTART"] = TD_data.groupby("ID").shift(1).loc[:,"daysB"]
  TD_data = TD_data[TD_data.TSTART.notna()]
  new_data = new_data.drop_duplicates(subset="ID",keep="first")
  return(new_data, TD_data)

# FoxInsight

##Enroll Date Estimation

In [ ]:
#Now that we have proof of concept, I will attempt to recreate the dataset using raw data from FOX
#First we need to establish a timeline (these files are large, so comment out to save time)
RPD = pd.read_csv(f"{datafolder}/FOX/ReturnPD.csv")

#Processing User data
U = pd.read_csv(f"{datafolder}/FOX/Users.csv")
U = U.loc[U.InitPDDiag == 1].drop("InitPDDiag",axis=1) #Keep only participants with PD at baseline

#Age and days_elapsed
RPD = RPD.loc[RPD.fox_insight_id.isin(U.fox_insight_id)] #Keep only participants with PD at baseline based on U

#To calculate baseline age (which will be used to identify days since baseline) we pull from Return PD
BLA = RPD.loc[:,["fox_insight_id","age","days_elapsed","schedule_of_activities"]]
BLA = BLA.loc[BLA.schedule_of_activities != "REG",:]
BLA.loc[:,"schedule_of_activities"] = BLA.schedule_of_activities.astype(float) #Convert SOA to float, this will be used to estimate age at baseline

BLA.loc[:,"BL_AGE"] = (BLA.age - (BLA.schedule_of_activities / 12)).round(2) #Calculate BL_AGE for each visit
n_na_BLA = BLA[BLA.BL_AGE.isna()].fox_insight_id.nunique()
BLA = BLA.dropna(subset = ["BL_AGE"]) #Remove NA values
print(f"{n_na_BLA} participants removed due to NA age at baseline")

#calculate baseline age and date, as well as registration age and date (sometimes the same, sometimes different)
BLA = BLA.pivot_table(index="fox_insight_id", values = ["days_elapsed","schedule_of_activities","BL_AGE"],
                      aggfunc = {"days_elapsed":"min","schedule_of_activities":"min","BL_AGE":"mean"})
BLA.loc[:,"BL_DT"] = BLA.days_elapsed - (BLA.schedule_of_activities * 30)
BLA = BLA.loc[:,["BL_AGE","BL_DT"]]
BLA = BLA.join(U.set_index("fox_insight_id").loc[:,["AgeAtEnrollment","days_elapsed"]]).rename({"AgeAtEnrollment":"REG_AGE","days_elapsed":"REG_DT"},axis=1)

#Save key for later use
k = BLA.loc[:,["BL_AGE","BL_DT"]].copy()

#Rewrite ReturnPD df including new variables
RPD = BLA.join(RPD.loc[:,["fox_insight_id","days_elapsed"]].set_index("fox_insight_id"))
RPD.loc[:,"daysB"] = RPD.days_elapsed - RPD.BL_DT #calculate time since baseline

#Some participants with changing diagnosis have negative daysB, so we remove these participants
ng_db = RPD[RPD.daysB < -90].index #We allow for a period of -90 to account for any time window abnormalities
RPD = RPD[~RPD.index.isin(ng_db)]#Remove NA values
print(f"{ng_db.nunique()} participants removed due to negative days since baseline")

'''
Return PD and Users data were obtained on 4/18/2022. Up until September 2019, participants were able
to complete their visits within a 90 day window (post this the window was 30 days). However, despite
this the majority of participants completed their surveys within 30 days (see ReturnPD days_acquired).
Based on this information, the earliest possible date of any last visit in this dataset would be
1/18/22 and we can estimate the earliest enrollment date.
'''
#In this case max db refers to the maximum days since baseline, in other words the latest possible visit for a participant
max_db = RPD.pivot_table(index = "fox_insight_id", values = "daysB", aggfunc = "max")

#We assume a normal distribution of visit dates, so we can estimate start date within a 95%
max_db.loc[:,"START_DT"] = [pd.to_datetime("3/1/22") - dt.timedelta(days = d) for d in max_db.daysB]

RPD = RPD.join(max_db.loc[:,"START_DT"])

RPD.describe()

In [ ]:
#Histogram of Estimated Enrollment Dates
print(f"{max_db.index.nunique()} participants total")
hist = px.histogram(max_db, x = "START_DT", height = 750, width = 1000, nbins = 20,
                    title = "Histogram of Estimate Start Date", labels = {'START_DT':"Estimated Date of Baseline",'y':"Number of Participants"}, text_auto=True)
hist.layout.yaxis.title.text = "Number of Participants"
# hist.data[0].texttemplate = "%{x}"
hist

In [ ]:
#What year-long enrollment period collects the most participants?
max = 0
for year in [2016,2017,2018,2019,2020,2021,2022]:
  for month in range(1,13,3):
    n = ((dt.datetime(year=year,month=month,day=1) < max_db.loc[:,"START_DT"]) & (max_db.loc[:,"START_DT"] < dt.datetime(year=year+1, month=month,day = 1))).sum()
    if n > max:
      print(f"The new max enroll start date is {month}/{year} with {n} participants")
      max = n

##Determine Inactivity
To determine inactivity, we pull data from all surveys administered to PD participants at a frequency of every six months or more often.

In [ ]:
#Load longitudinal data last collected on 4/20
# L = pd.read_csv(f"{datafolder}/FOX/LONG-420.csv")
# L = L[L.fox_insight_id.isin(k.index)]
# k1 = k.join(L.set_index("fox_insight_id"))

'''
To determine inactivity, we cycle through all of the surveys administered on a regular basis by Fox
Insight and identify the latest value of days_elapsed for each participant. Comparing this value to
the value of maximum possible daysB established by the ReturnPD survey allows us to establish a
period of activity and inactivity.
'''

k1 = pd.DataFrame()
for csv in os.listdir(f"{datafolder}/FOX/"):
  if "csv" in csv:
    df = pd.read_csv(f"{datafolder}/FOX/{csv}")
    if ("days_elapsed" in df.columns) & ~(csv in ["ReturnPD.csv","Users.csv","About.csv"]): #These three have different definitions for days_elapsed
      df = df.loc[:,["fox_insight_id","days_elapsed"]]
      k1 = k1.append(df).drop_duplicates().sort_values(["fox_insight_id","days_elapsed"])
# pd.read_csv(f"{datafolder}/FOX/Medications.csv")

k1 = k.join(k1.set_index("fox_insight_id"))

#Round negative values of daysB to 0, we already cleaned far outliers in previous chunk, so this just finishes the job
k1.loc[:,"daysB"] = [0 if d < 0 else d for d in (k1.days_elapsed - k1.BL_DT)]
k1.loc[:,"VM"] = (k1.daysB / 90).round(0) * 3


k1 = (k1.join(k1.reset_index()
              .drop_duplicates(subset = ["fox_insight_id","VM"],keep="first")
              .pivot_table(index=["fox_insight_id"],values=["VM"],aggfunc ="count")
              .rename({"VM":"NVISITS"},axis=1)))

k1 = k1.join((k1.reset_index().loc[(k1.VM <= 24).values]
                 .drop_duplicates(subset = ["fox_insight_id","VM"],keep="first")
                 .pivot_table(index=["fox_insight_id"],values=["VM"],aggfunc ="count")
                 .rename({"VM":"NVISITS24"},axis=1)))

k1 = k1[~k1.index.duplicated(keep = "last")]

k1.loc[:,"ACT"] = k1.days_elapsed - k1.BL_DT

k1 = max_db.rename({"daysB":"max_daysB"},axis=1).join(k1)

k1.loc[:,"INACT"] = k1.max_daysB - k1.ACT

k1.loc[:,"DROP_DT"] = [st + dt.timedelta(days = dat) if ina > 90 else np.nan for st,dat,ina in zip(k1.START_DT,k1.ACT,k1.INACT)]

k1

In [ ]:
px.histogram(k1.DROP_DT,nbins = 20)

In [ ]:
k1.INACT.describe()

In [ ]:
for thresh in range(3,24,3):
  print(f"{((k1.ACT / 30 > thresh).sum() / len(k1.ACT) * 100).round(2)} percent attrition with threshold of {thresh}")
px.histogram(k1.ACT, nbins = int(k1.ACT.max() / 90))

## Produce covariates

In [ ]:
'''
This section involves remapping covariates for analysis. Here we use the raw var to load onto the
cov dataframe. cov dataframe relies on key created in previous code chunks
'''
cov = k.copy()
cov.loc[:,"BL_AGE"] = cov.BL_AGE.round(1)

#From about file, we can pull race, ethnicity, income, employment, veteran status, and previous research involvement
raw = (pd.read_csv(f"{datafolder}/FOX/About.csv")
        .drop(["age","days_elapsed","days_acquired","schedule_of_activities","HeightPNA",
               "HeightInch","HeightCm","WeightPNA","WeightKgs","WeightLbs"], axis=1)
        .drop_duplicates(subset = "fox_insight_id", keep="first"))
raw = raw.loc[raw.fox_insight_id.isin(k.index),:]

#Process Sex
raw.loc[:,"SEX"] = raw.Sex.map({1:"Male",2:"Female"})
raw = raw.drop("Sex",axis=1)

#Process Ethnicity
raw.loc[:,"ETHN"] = [ "HISP" if (nh==1) and (m==p==c==l==0)
                      else "NONH" if (nh==0) and (m+p+c+l)>0
                      else np.nan for (nh,m,p,c,l) in zip( raw['EthnNotHispanic'], raw['EthnMexican'], raw['EthnPuerto'], raw['EthnCuban'], raw['EthnLatino'])]
raw = raw.drop(raw.columns[raw.columns.str.contains("Ethn")],axis=1)

#Process Race
raw.loc[:,"RACE"] = ['HISP' if (w == 1) and (hs == "HISP")
                    else 'WHIT' if (w==1) and (aa==ai==a==nh==0)
                    else 'BLCK' if (aa==1) and (w==ai==a==nh==0)
                    else 'ASIA' if (a==1) and (w==ai==aa==nh==0)
                    else 'OTH' if (w+aa+ai+a+nh)>0
                    else np.nan for (w,aa,ai,a,nh,hs) in zip(
                    raw['RaceW'], raw['RaceAA'], raw['RaceAI'], raw['RaceA'], raw['RaceNH'], raw["ETHN"])]
raw = raw.drop(["RaceAA","RaceAI","RaceA","RaceNH","RacePNA"],axis=1)



#Process education as a categorical with four levels; <HS (Less than High School),HS (High School or Some College),
# UNI (Bachelor's or Associate),UNI+ (Doctorate or Master's)
raw.loc[:,"EDUC"] = ["<HS" if e == 1
                           else "HS" if e in [2,3]
                           else "UNI" if e in [4,5]
                           else "UNI+" if e in [6,7,8]
                           else np.nan for e in raw.Education]

raw.loc[:,"EDUCYRS"] = raw.Education.map({1:8, 2:12, 3:12, 4:14, 5:16, 6:18, 7:20, 8:20})
raw = raw.drop("Education",axis=1)



#Process income as a categorical with 3 levels; <50k, 50-100k, >100k
raw.loc[:,"INCM3"] = [0 if i in [1,2,3]
                       else 1 if i in [4,5]
                       else 2 if i == 6
                       else np.nan for i in raw.Income]
raw = raw.rename({"Income":"INCM7"},axis=1) #We also preserve the initial income coding as income with 7 levels

#Employment status as a categorical with fulltime, parttime, retired, and unemployed status
raw.loc[:,"EMPL"] = ["EMP" if e in [1,2]
                     else "RET" if e == 3
                     else "UNEMP" if e == 4
                     else np.nan for e in raw.Employment]
raw = raw.drop("Employment",axis=1)

#Add combined employment/income outcome
raw.loc[:,"SES"] = ["RET" if e == "RET"
                        else "UNEMP" if e == "UNEMP"
                        else "EMP>50" if i in [1,2]
                        else "EMP<50" if i == 0
                        else np.nan for i,e in zip(raw.INCM3,raw.EMPL)]

#Veteran and Research status require minimal cleaning
raw.loc[:,"VET"] = raw.Veteran.map({0:0,1:1,3:np.nan})
raw.loc[:,"RSRC"] = raw.Research.map({0:0,1:1,3:np.nan})
raw = raw.drop(["Veteran","Research"],axis=1)

#Join demographics to key
cov = cov.join(raw.set_index("fox_insight_id"))

raw = pd.read_csv(f"{datafolder}/FOX/Users.csv").loc[:,["fox_insight_id","LocCountry","InitPDDiagAge"]]

raw = (raw.loc[raw.fox_insight_id.isin(k.index),:]
          .rename({"LocCountry":"LOC","InitPDDiagAge":"BL_DDX"},axis=1)
          .set_index("fox_insight_id"))

cov = cov.join(raw)

cov.loc[:,"BL_DDX"] = cov.BL_AGE - cov.BL_DDX

#Measure of ADLs is captured in "Your Movement Experiences" Survey (UPDRS2)
raw = pd.read_csv(f"{datafolder}/FOX/Movement.csv").drop_duplicates(subset="fox_insight_id",keep="first")
UP2_ITEMS = raw.columns[raw.columns.str.contains("Move")][1:]
UP2_ITEMS = dict(zip(UP2_ITEMS, [i.replace("Move","UP2_").upper()for i in UP2_ITEMS]))
raw.loc[:,"BL_UP2"] = raw.loc[:,UP2_ITEMS.keys()].sum(axis=1, skipna=False)
raw.loc[:,"PROXY"] = raw.MoveWho.replace({1:0,2:1,3:1})#.replace({1:"PT",2:"CG",3:"PT/CG"})
raw = raw.rename(UP2_ITEMS, axis=1)
raw = raw.loc[:,list(UP2_ITEMS.values()) + ["fox_insight_id","BL_UP2","PROXY"]].set_index("fox_insight_id")

cov = cov.join(raw)

# #Measure of Motor Function is captured in "Brief Motor Screen" Survey
# raw = pd.read_csv(f"{datafolder}/FOX/Brief_Motor_Screen.csv")
# raw.loc[:,"FID"] = raw.fox_insight_id
# raw = raw.loc[:,raw.columns[raw.columns.str.contains("id|MtrScrn",case=False)]]
# raw = raw.groupby("FID").fillna(method="backfill")
# raw = raw.drop_duplicates(subset="fox_insight_id",keep="first")
# raw.loc[:,"BL_BMS"] = raw.loc[:,raw.columns[raw.columns.str.contains("MtrScrn")]].replace({2:np.nan}).sum(axis=1, skipna=True)
# raw = raw.loc[:,["fox_insight_id","BL_BMS"]].set_index("fox_insight_id")
# raw

# cov = cov.join(raw)

# Living status is reported in the "Return PD" survey
# This presented a problem because it conflicted with attrition definition significnatly
RPD = pd.read_csv(f"{datafolder}/FOX/ReturnPD.csv").set_index("fox_insight_id")

RPD.loc[:,"LIVSUM"] = RPD.loc[:,RPD.columns.str.contains("Live")].sum(axis=1).replace({0:np.nan})
RPD = RPD.dropna(subset = ["LIVSUM"])
RPD = RPD.loc[~RPD.index.duplicated(keep="first")].drop("LIVSUM",axis=1)

RPD.loc[:,"LIV"] = ["wspouse" if sp == 1
                    else "wfam"  if (ach + mch + of) > 0
                    else "wcare" if ct == 1
                    else "@ass" if (ass + nh) > 0
                    else "alone" if a == 1
                    else np.nan for a,sp,ach,mch,of,ct,ass,nh in zip(RPD.LiveAlonePD,RPD.LiveSpousePD,
                                                             RPD.LiveAdultPD, RPD.LiveMinorPD,
                                                             RPD.LiveOthFamPD, RPD.LiveCarePD,
                                                             RPD.LiveAsstPD, RPD.LiveNursPD)]
RPD.loc[:,"LIVASS"] = [1 if (al + nh) > 0 else 0 for al, nh in zip(RPD.LiveAsstPD, RPD.LiveNursPD)]
RPD.loc[:,"LIVFAM"] = [1 if (ach + mch + of + sp) > 0 else 0 for sp,ach,mch,of in zip(RPD.LiveSpousePD,
                                                             RPD.LiveAdultPD, RPD.LiveMinorPD,
                                                             RPD.LiveOthFamPD, )]



RPD = RPD.loc[:,["LIV","LIVASS","LIVFAM","LiveAlonePD","LiveCarePD"]]

cov = cov.join(RPD)

#Measure of Cognitive Impairment is captured in "Your Cognition and Daily Activities" Survey (PDAQ)
raw = pd.read_csv(f"{datafolder}/FOX/DailyActivity.csv").drop_duplicates(subset="fox_insight_id",keep="first")
raw.loc[:,"BL_PDAQ"] = raw.loc[:,raw.columns[raw.columns.str.contains("Daily")]].sum(axis=1, skipna=False)
raw.loc[:,"BL_MCI"] = [1 if p <= 43 else 0 for p in raw.BL_PDAQ]
raw.loc[:,"BL_DEM"] = [1 if p <= 37 else 0 for p in raw.BL_PDAQ]
raw.loc[:,"BL_COG"] = [2 if p <= 37 else 1 if p <= 47 else 0 for p in raw.BL_PDAQ]
raw = raw.loc[:,["fox_insight_id","BL_PDAQ","BL_MCI","BL_DEM","BL_COG"]].set_index("fox_insight_id")

cov = cov.join(raw)

#Measure of Depressive Symptoms is captured in "Your Mood" Survey (GDS-15)
raw = pd.read_csv(f"{datafolder}/FOX/Mood.csv").drop_duplicates(subset="fox_insight_id",keep="first")
raw.loc[:,["MoodSatis","MoodSpirits","MoodHappy","MoodAlive","MoodEnergy"]] = raw.loc[:,["MoodSatis","MoodSpirits","MoodHappy","MoodAlive","MoodEnergy"]].replace({0:1,1:0})
raw.loc[:,"BL_DEP"] = raw.loc[:,raw.columns[raw.columns.str.contains("Mood")]].sum(axis=1,skipna=True)
raw.loc[:,"IS_DEP"] = [1 if d > 5 else 0 if d < 5 else 0 for d in raw.BL_DEP]
raw = raw.loc[:,["fox_insight_id","BL_DEP","IS_DEP"]].set_index("fox_insight_id")

cov = cov.join(raw)

#Measure of physical activity is captured in "Your Physical Activities"
raw = pd.read_csv(f"{datafolder}/FOX/PASE.csv").drop_duplicates(subset="fox_insight_id",keep="first")

PASE_dict = dict(zip(raw.columns[7:],["Q2","Q2a","Q3","Q3a","Q4","Q4a","Q5","Q5a","Q6","Q6a",
                          "Q7","Q8","Q9a","Q9b","Q9c","Q9d","Q10","Q10a"]))
raw = raw.rename(PASE_dict,axis=1).replace({np.nan:0})
wts = dict(zip(["Q2","Q3","Q4","Q5","Q6","Q7","Q8","Q9a","Q9b","Q9c","Q9d","Q10"],
                   [20,21,23,23,30,25,25,30,36,20,35,21]))

HPDqs = ["Q2","Q3","Q4","Q5","Q6"]
for q in HPDqs:
  raw.loc[:,f"{q}t"] = [wts[q] * .11 if  (qd == 1) & (qh == 1)
                        else wts[q] * .32 if  (qd == 1) & (qh == 2)
                        else wts[q] * .64 if  (qd == 1) & (qh == 3)
                        else wts[q] * 1.07 if (qd == 1) & (qh == 4)
                        else wts[q] * .25 if  (qd == 2) & (qh == 1)
                        else wts[q] * .75 if  (qd == 2) & (qh == 2)
                        else wts[q] * 1.50 if  (qd == 2) & (qh == 3)
                        else wts[q] * 2.50 if (qd == 2) & (qh == 4)
                        else wts[q] * .43 if  (qd == 3) & (qh == 1)
                        else wts[q] * 1.29 if  (qd == 3) & (qh == 2)
                        else wts[q] * 2.57 if  (qd == 3) & (qh == 3)
                        else wts[q] * 4.29 if (qd == 3) & (qh == 4)
                        else 0 for qd, qh in zip(raw.loc[:,q],raw.loc[:,f"{q}a"])]
raw = raw.drop(HPDqs + [f"{q}a" for q in HPDqs], axis=1)

qset2 = ["Q7","Q8","Q9a","Q9b","Q9c","Q9d"]
for q in qset2:
  raw.loc[:,f"{q}t"] = [n * wts[q] for n in raw.loc[:,q]]
raw.drop(qset2,axis=1)

raw.loc[:,"Q10t"] = [21 if (ten == 1) * (tena > 1) else 0 for ten, tena in zip(raw.Q10,raw.Q10a)]

raw.loc[:,"LEIS"] = raw.LeisureDay * raw.LeisureHours

raw.loc[:,"PASE"] = raw.loc[:,raw.columns[raw.columns.str.contains("t$")]].sum(axis=1)

raw = raw.set_index("fox_insight_id").loc[:,["LEIS","PASE"]]

cov = cov.join(raw)

cov

## Produce final dataset

### Create Timeline Objects

In [ ]:
enroll_dts =  k1.loc[:,"START_DT"]


tl1 = ((dt.datetime(year=2017,month=10,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=2018, month=10,day = 1)))
print(f'{tl1.sum()} participants in max enroll timeline')
tl2 = ((dt.datetime(year=2017,month=3,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=2018, month=3,day = 1)))
print(f'{tl2.sum()} participants in pre-COVID timeline')
tl3 = ((dt.datetime(year=2019,month=3,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=2020, month=3,day = 1)))
print(f'{tl3.sum()} participants in COVID follow-up timeline')
tl4 = ((dt.datetime(year=2020,month=3,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=2021, month=3,day = 1)))
print(f'{tl4.sum()} participants in COVID recruitment timeline')
tl5 = ((dt.datetime(year=2019,month=9,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=2020, month=9,day = 1)))
print(f'{tl5.sum()} participants in post time window change timeline')
tl6 = ((dt.datetime(year=2018,month=10,day=2) < enroll_dts) & (enroll_dts < dt.datetime(year=2019, month=10,day = 1)))
print(f'{tl6.sum()} participants in year 2 recruitment timeline')

#Now we can produce our complete dataset and filter based on dates of activity
#Set parameters
tlps = [tl1[tl1].index, tl2[tl2].index, tl3[tl3].index, tl4[tl4].index, tl5[tl5].index,tl6[tl6].index]
tlns = [f"TL{i}" for i in range(1,7)]
FUPs = [24,48,12,12,15,24]

#Create dataframes
for tlp, tln, FUP in zip(tlps, tlns, FUPs):
  print(f"************Running {tln} with follow-up of {FUP} months************")
  cdat = k1.drop(["BL_AGE","BL_DT"],axis=1).join(cov)
  cdat = cdat[cdat.index.isin(tlp)]
  cdat.insert(2, "LTF", [0 if act > (FUP * 30) else 1 for act in cdat.ACT])
  cdat.loc[:,"daysB"] = [(FUP * 30) if act >= (FUP * 28) else act for act in cdat.ACT]

  covars = ["daysB","BL_AGE","BL_DDX","SEX","RSRC","RACE", "ETHN", "EDUC", "INCM7", "INCM3", "EMPL", "VET",
        "BL_UP2","BL_COG","BL_DEP","PROXY","PASE"]

  print(f"{cdat.index.nunique()} participants before cleaning")
  for col in covars:
    print(f"dropping {cdat[cdat.loc[:,col].isna()].index.nunique()} participants due to {col}")
    cdat = cdat.dropna(subset = [col])
  npart = cdat.index.nunique()
  print(f"{npart} participants after cleaning")

  cdat.loc[:,"UNOS"] = [1 if nv == 1 else 0 for nv in cdat.NVISITS24]

  print(f"{cdat[cdat.UNOS == 1].index.nunique()} participants with only one visit ({round(cdat[cdat.UNOS == 1].index.nunique() * 100 / npart, 1)}%)")

  cdat.loc[:,"NONCOMP"] = [0 if ((nv / ((FUP / 3) + 1))) > 0.5 else 1 for nv in cdat.NVISITS24]

  n_ncmp = cdat[cdat.NONCOMP == 1].index.nunique()
  print(f"{n_ncmp} compliant participants ({round(n_ncmp * 100 / npart, 1)}%)")

  cdat.to_csv(f"FOX_{tln}_LTF{FUP}.csv")
  print(f'{cdat[cdat.LTF == 1].index.nunique()} participants lost to follow up ({round(cdat[cdat.LTF == 1].index.nunique() * 100 / npart, 1)}%)')

In [ ]:
tl1 = pd.read_csv("FOX_TL1_LTF24.csv")
print("***Compliance and Attrition Parameters for Returning Participants of TL1****")
print(tl1[tl1.UNOS == 0].NONCOMP.value_counts())
print(tl1[tl1.UNOS == 0].LTF.value_counts())

In [ ]:
tl1 = pd.read_csv("FOX_TL6_LTF24.csv")
print("***Noncompliance and Attrition Parameters for Returning Participants of TL6****")
print(tl1[tl1.UNOS == 0].NONCOMP.value_counts())
print(tl1[tl1.UNOS == 0].LTF.value_counts())

In [ ]:
cov.reset_index().pivot_table(index = "EMPL",columns = "INCM",values = "fox_insight_id", aggfunc = "count")

### First Year Timeline


#### Return Participation

In [ ]:
%%R
#Loads dataset created in python
LTFdf = fread("FOX_TL1_LTF24.csv")
LTFdf$RACE <- relevel(factor(LTFdf$RACE), ref = "WHIT")
LTFdf$EDUC <- relevel(factor(LTFdf$EDUC), ref = "HS")
# LTFdf$PROXY <- relevel(factor(LTFdf$PROXY), ref = "PT")

print("******************COMPARING ONE-TIMERS AND RETURNERS*****************")
RTLM_Y1 <- glm(formula = UNOS  ~ (SEX + I(BL_AGE/5) + RACE + EDUC + factor(INCM3) + EMPL + RSRC + PASE + PROXY + I(BL_DDX/5) + BL_UP2 + factor(BL_COG) + IS_DEP),
             data = LTFdf, family = "binomial")
print(summary(RTLM_Y1))
print(exp(coef(RTLM_Y1)))

In [ ]:
%%R
LTFdf = fread("FOX_TL1_LTF24.csv")

probabilities <- predict(RTLM_Y1, type = "response")
predicted.classes <- ifelse(probabilities > 0.5, "pos", "neg")

mydata <- LTFdf %>%
         select(c("BL_AGE","BL_DDX","BL_UP2"))
predictors <- colnames(mydata)
mydata <- mydata %>%
         mutate(logit = log(probabilities/(1-probabilities))) %>%
         gather(key = "predictors", value = "predictor.value", -logit)


ggplot(mydata, aes(logit, predictor.value))+
  geom_point(size = 0.5, alpha = 0.5) +
  geom_smooth(method = "loess") +
  theme_bw() +
  facet_wrap(~predictors, scales = "free_y")

#### Nonadherence


In [ ]:
%%R
print("******************NONCOMPLIANCE ANALYSIS*****************")
LTFdf = fread("FOX_TL1_LTF24.csv")

NCLM_Y1 <- glm(formula = NONCOMP ~ (SEX + I(BL_AGE/5) + RACE + EDUC + factor(INCM3) + EMPL + RSRC + PASE + PROXY + I(BL_DDX/5) + BL_UP2 + factor(BL_COG) + IS_DEP),
             data = filter(LTFdf, UNOS == 0), family = "binomial")
print(summary(NCLM_Y1))
print(exp(coef(NCLM_Y1)))

In [ ]:
%%R
LTFdf = fread("FOX_TL1_LTF24.csv")

probabilities <- predict(NCLM_Y1, type = "response")
predicted.classes <- ifelse(probabilities > 0.5, "pos", "neg")

mydata <- LTFdf %>%
         filter(UNOS == 0) %>%
         select(c("BL_AGE","BL_DDX","BL_UP2"))
predictors <- colnames(mydata)
mydata <- mydata %>%
         mutate(logit = log(probabilities/(1-probabilities))) %>%
         gather(key = "predictors", value = "predictor.value", -logit)


ggplot(mydata, aes(logit, predictor.value))+
  geom_point(size = 0.5, alpha = 0.5) +
  geom_smooth(method = "loess") +
  theme_bw() +
  facet_wrap(~predictors, scales = "free_y")

#### Cox LTF

In [ ]:
%%R
print("******************MAX ENROLLMENT*****************")
LTFdf = fread("FOX_TL1_LTF24.csv")
LTFdf$RACE <- relevel(factor(LTFdf$RACE), ref = "WHIT")

LTF_Y1 <- coxph(formula = Surv(daysB, LTF == 1) ~ (SEX + I(BL_AGE/5) + RACE + EDUC + factor(INCM3) + EMPL + RSRC + PASE + PROXY + I(BL_DDX/5) + BL_UP2 + factor(BL_COG) + IS_DEP),
             data = filter(LTFdf, UNOS == 0))
res = cbind(exp(coef(LTF_Y1)), exp(confint(LTF_Y1)), coef(summary(LTF_Y1))[,5])
colnames(res) <- c("OR","CI2.5%","CI97.5%","P")
res <- signif(res, digits=3)


g <- ggsurvplot(fit = survfit(Surv(daysB, LTF == 1) ~ 1, data = filter(LTFdf, UNOS == 0)),
    xlab = "Days Since Baseline",
    ylab = "Probability of Patient Remaining in Study",
    title = "Likelihood of Attrition in Fox Insight Year One",
    legend.title = "",
    risk.table.y.text = F,
    risk.table = TRUE)

print(summary(LTF_Y1))
print(res)
print(g)

#### UP2

In [ ]:
%%R
print("******************MAX ENROLLMENT*****************")
LTFdf = fread("FOX_TL1_LTF24.csv")
LTFdf$RACE <- relevel(factor(LTFdf$RACE), ref = "WHIT")

UP2_Y1 <- coxph(formula = Surv(daysB, LTF == 1) ~ UP2_SPEECH+UP2_SALIVA+UP2_CHEW+UP2_EAT+UP2_DRESS+UP2_HYGIENE+UP2_WRITE+UP2_HOBBY+UP2_SLEEP+UP2_TREMOR+UP2_UP+UP2_WALK+UP2_FREEZE,
             data = filter(LTFdf, UNOS == 0))
res = cbind(exp(coef(UP2_Y1)), exp(confint(UP2_Y1)), coef(summary(UP2_Y1))[,5])
colnames(res) <- c("OR","CI2.5%","CI97.5%","P")
res <- signif(res, digits=3)

print(summary(UP2_Y1))
print(res)

In [ ]:
%%R
library(table1, lib.loc = pack)
LTFdf = fread("FOX_TL1_LTF24.csv")

t = table1(~ (SEX + BL_AGE + RACE + EDUC + factor(INCM3) + EMPL + factor(RSRC) + BL_DDX + BL_UP2 + factor(BL_COG) + factor(IS_DEP) + factor(NONCOMP) + factor(LTF))
           , data = LTFdf, render.continuous="Median [Q1,Q3]")

gsub("\n", "", t[1])

#### Restricted Analysis for Comparison to PPMI

In [ ]:
%%R
print("******************MAX ENROLLMENT*****************")
LTFdf = fread("FOX_TL1_LTF24.csv")
LTFdf$RACE <- relevel(factor(LTFdf$RACE), ref = "WHIT")

FIR_Y1 <- coxph(formula = Surv(daysB, LTF == 1) ~ (SEX + I(BL_AGE/5) + I(RACE != "WHIT") + EDUC + I(BL_DDX/5) + BL_UP2 + factor(BL_COG) + IS_DEP),
             data = filter(LTFdf, UNOS == 0))
res = cbind(exp(coef(FIR_Y1)), exp(confint(FIR_Y1)), coef(summary(FIR_Y1))[,5])
colnames(res) <- c("OR","CI2.5%","CI97.5%","P")
res <- signif(res, digits=3)

print(summary(FIR_Y1))
print(res)
print(g)

### Replication Timeline

#### Return Participation

In [ ]:
%%R
#Loads dataset created in python
LTFdf = fread("FOX_TL6_LTF24.csv")
LTFdf$RACE <- relevel(factor(LTFdf$RACE), ref = "WHIT")

print("******************COMPARING ONE-TIMERS AND RETURNERS*****************")
RTLM_Y2 <- glm(formula = UNOS  ~ (SEX +I(BL_AGE/5) + RACE + EDUC + factor(INCM3) + EMPL  + RSRC + I(BL_DDX/5) + BL_UP2 + factor(BL_COG) + IS_DEP),
             data = LTFdf, family = "binomial")
print(summary(RTLM_Y2))

#### Nonadherence

In [ ]:
print("******************NONCOMPLIANCE ANALYSIS*****************")
NCLM_Y2 <- glm(formula = NONCOMP ~ (SEX + I(BL_AGE/5) + RACE + EDUC + factor(INCM3) + EMPL  + RSRC + I(BL_DDX/5) + BL_UP2 + factor(BL_COG) + IS_DEP),
             data = filter(LTFdf, UNOS == 0), family = "binomial")
print(summary(NCLM_Y2))


#### Cox LTF

In [ ]:
%%R
print("******************MAX ENROLLMENT*****************")
LTFdf = fread("FOX_TL6_LTF24.csv")
LTFdf$RACE <- relevel(factor(LTFdf$RACE), ref = "WHIT")

LTF_Y2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (SEX + I(BL_AGE/5) + RACE + EDUC + factor(INCM3) + EMPL + RSRC + I(BL_DDX/5) + BL_UP2 + factor(BL_COG) + IS_DEP),
             data = filter(LTFdf, UNOS == 0))
res = cbind(exp(coef(LTF_Y2)), exp(confint(LTF_Y2)), coef(summary(LTF_Y2))[,5])
colnames(res) <- c("OR","CI2.5%","CI97.5%","P")
res <- signif(res, digits=3)


g <- ggsurvplot(fit = survfit(Surv(daysB, LTF == 1) ~ 1, data = filter(LTFdf, UNOS == 0)),
    xlab = "Days Since Baseline",
    ylab = "Probability of Patient Remaining in Study",
    title = "Likelihood of Attrition in Fox Insight Year Two",
    legend.title = "",
    risk.table.y.text = F,
    risk.table = TRUE)

print(summary(LTF_Y2))
print(res)
print(g)

#### UP2

In [ ]:
%%R
print("******************MAX ENROLLMENT*****************")
LTFdf = fread("FOX_TL6_LTF24.csv")
LTFdf$RACE <- relevel(factor(LTFdf$RACE), ref = "WHIT")

UP2_Y2 <- coxph(formula = Surv(daysB, LTF == 1) ~ UP2_SPEECH+UP2_SALIVA+UP2_CHEW+UP2_EAT+UP2_DRESS+UP2_HYGIENE+UP2_WRITE+UP2_HOBBY+UP2_SLEEP+UP2_TREMOR+UP2_UP+UP2_WALK+UP2_FREEZE,
             data = filter(LTFdf, UNOS == 0))
res = cbind(exp(coef(UP2_Y2)), exp(confint(UP2_Y2)), coef(summary(UP2_Y2))[,5])
colnames(res) <- c("OR","CI2.5%","CI97.5%","P")
res <- signif(res, digits=3)

print(summary(UP2_Y2))
print(res)

In [ ]:
%%R
library(table1, lib.loc = pack)

t = table1(~ (SEX + BL_AGE + RACE + EDUC + factor(INCM3) + EMPL + factor(RSRC) + BL_DDX + BL_UP2 + factor(BL_COG) + factor(IS_DEP) + factor(NONCOMP) + factor(LTF))
           , data = LTFdf, render.continuous="Median [Q1,Q3]")

gsub("\n", "", t[1])

### 1st & 2nd Year Meta-Analysis

In [ ]:
%%R
results_fox

In [ ]:
%%R
pack <- "/content/gdrive/My Drive/R/packages"
library(metafor, lib.loc = pack)

Y1_res <- c(summary(LTF_Y1)$coef[,1],summary(LTF_Y1)$coef[,3])
Y2_res <- c(summary(LTF_Y2)$coef[,1],summary(LTF_Y2)$coef[,3])

results_fox <- data.frame(rbind(Y1_res, Y2_res))
num_vars <- (length(names(results_fox)) / 2)
for (i in 1:(num_vars-1)){
    model2 = rma(results_fox[[i]], sei=results_fox[[i + num_vars]], slab=c("Y1","Y2"), method='REML')
    print("***************************")
    print(names(results_fox)[i])
    print(model2)
    print("HR")
    print(exp(model2$b))
    print(exp(model2$ci.lb))
    print(exp(model2$ci.ub))

}

### Ad-Hoc PPMI Analysis

In [ ]:
#Loads harmonized PPMI data from GP2
PPMI = pd.read_csv(f'{datafolder}/PPMI_GP2.csv')

#Loads info on in or out status
PS = pd.read_csv(f'{datafolder}/Participant_Status.csv')
PS.loc[:,"ENROLL_STATUS"] = PS.ENROLL_STATUS.str.lower()

PS = PS.loc[PS.ENROLL_STATUS.isin(["enrolled","withdrew","declined","complete"])]

PS.loc[:,"ENROLL_DATE"] = pd.to_datetime(PS.ENROLL_DATE)

#Create "Loss to Follow-Up" Variable, then subset data
PS.loc[:,"censure_date"] = (pd.to_datetime(PS.STATUS_DATE) - pd.Timestamp("1970-01-01")) // pd.Timedelta("1D")

PS.loc[:,"days_to_cens"] = (pd.to_datetime(PS.STATUS_DATE) - pd.to_datetime(PS.ENROLL_DATE)) // pd.Timedelta("1D")
PS.loc[:,"LTF2"] = [1 if ((s in ["withdrew","declined"]) & (d <= 730)) else 0 for s,d in zip(PS.ENROLL_STATUS,PS.days_to_cens)]
PS.loc[:,"LTF5"] = [1 if ((s in ["withdrew","declined"]) & (d <= 1825)) else 0 for s,d in zip(PS.ENROLL_STATUS,PS.days_to_cens)]

PS = PS.loc[:,["PATNO","COHORT","censure_date","LTF2","LTF5"]]

PPMI = PPMI.merge(PS, left_on =  "participant_id", right_on = "PATNO")

PPMI = PPMI[PPMI.Phenotype.isin(["Control","PD"])]

PPMI.loc[:,"ethrac"] = ["HISP" if e == "Hispanic or Latino"
                      else "ASIA" if r == "Asian"
                      else "BLAC" if r == "Black or African American"
                      else "WHIT" if r == "White"
                      else "OTH" if r in ["Multi-racial","Other","American Indian or Alaska Native"]
                      else np.nan for e,r in zip(PPMI.ethnicity,PPMI.race)]

PPMI.loc[:,"educlvl"] = ["<HS" if yr < 12
                         else "HS" if yr < 14
                         else "UNI" if yr < 18
                         else "UNI+" if yr >= 18
                         else np.nan for yr in PPMI.education_years]

PPMI.loc[:,"cog3"] = ["DEM" if mca < 18
                      else "MCI" if mca < 26
                      else "NORM" for mca in PPMI.moca_total_score]

LTF = "LTF2"
PPMI_cols = {"ID":"participant_id",
              "BL_DATE":'date_baseline',
              "DATE":'date_visit',
              "VM":'visit_month',
              "CENS_DT":"censure_date",
              "LTF":LTF,
              "PHENO":'Phenotype',
              'SEX':'sex',
              'RACE':"ethrac",
              "ETHN":"ethnicity",
              'STUDY_ARM':'study_arm',
              'EDUC':'educlvl',
              "EDUCYRS":"education_years",
              "DX_AGE":'age_at_diagnosis',
              'BL_AGE':'age_at_baseline',

              "UP1_COG":"code_upd2101_cognitive_impairment",
              "UP1_PSYC":"code_upd2102_hallucinations_and_psychosis",
              "UP1_DEP":"code_upd2103_depressed_mood",
              "UP1_ANX":"code_upd2104_anxious_mood",
              "UP1_APAT":"code_upd2105_apathy",

              "UP2_SPEEC": "code_upd2201_speech",
              "UP2_SALIV":"code_upd2202_saliva_and_drooling",
              "UP2_SWALL":"code_upd2203_chewing_and_swallowing",
              "UP2_EAT":"code_upd2204_eating_tasks",
              "UP2_DRESS":"code_upd2205_dressing",
              "UP2_HYGEI":"code_upd2206_hygiene",
              "UP2_HNDWR":"code_upd2207_handwriting",
              "UP2_HOBBY":"code_upd2208_doing_hobbies_and_other_activities",
              "UP2_BED":"code_upd2209_turning_in_bed",
              "UP2_TREM":"code_upd2210_tremor",
              "UP2_GETUP":"code_upd2211_get_out_of_bed_car_or_deep_chair",
              "UP2_WALK":"code_upd2212_walking_and_balance",
              "UP2_FREEZ":"code_upd2213_freezing",
              "UP2":'mds_updrs_part_ii_summary_score',
              "UP3":'mds_updrs_part_iii_summary_score',
              "APAT":"code_upd2105_apathy",
              "COG":'moca_total_score',
              'COG3':'cog3',
              "DEP":'depress_test_score',
              "IS_DEP":'depress_test_score/5',
  }


PPMI_CS, PPMI_LNG = PrepKM(PPMI, PPMI_cols)

  # PPMI_CRA = PPMI_CS.copy()
  # PPMI_CRA.loc[:,"CRA_MCI"] = [1 if mca < 26 else 2 if ltf == 1 else 0 for mca,ltf in zip(PPMI_CS.COG,PPMI_CS.LTF)]
  # PPMI_CRA.loc[:,"CRA_DEM"] = [1 if mca < 18 else 2 if ltf == 1 else 0 for mca,ltf in zip(PPMI_CS.COG,PPMI_CS.LTF)]
  # PPMI_CRA.loc[:,"CRA_DEP"] = [1 if dep == 1 else 2 if ltf == 1 else 0 for dep,ltf in zip(PPMI_CS.DEP,PPMI_CS.LTF)]

PPMI_CS.to_csv(f'PPMI_{LTF}_CS.csv', index=False)
PPMI_LNG.to_csv(f'PPMI_{LTF}_LNG.csv', index=False)

In [ ]:
%%R
PPMI_LTF2_CS <- fread("PPMI_LTF2_CS.csv")
# TI_LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (sex + education_years + BL_AGE + BL_UP2 + BL_UP3 + BL_MCA + BL_GDS), data = filter(kd, Phenotype == "PD"))
# TI_LTF <- coxph(formula = Surv(TSTART, daysB, LTF_TD == 1) ~ (EDUC + SEX + BL_AGE + BL_durdx + UP2 + UP3 + I(COG < 27) + I(DEP>4)), data = filter(PPMI_LNG, PHENO == "PD"))
# print(summary(TI_LTF))

PPMI2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (SEX + BL_AGE + EDUC + BL_durdx + BL_UP2 + BL_UP3 + I(BL_COG < 26) + I(BL_DEP>4)), data = filter(PPMI_LTF2_CS, PHENO == "PD"))
print(summary(PPMI2))



### Tables for First Two Years Analysis

In [ ]:
%%R
LMRes <- function(model){
  t <- coef(summary(model)) %>%
     as.data.frame(.)
  names(t) <- c("E","SE","Z","P")
  t <- t %>%
       mutate(ast = case_when(P < 0.001 ~ "***",
                         P < 0.01 ~ "**",
                         P < 0.05 ~ "*",
                         TRUE ~ ""),
              P = if_else(P<0.01, formatC(P, digits=1, format = "e"), as.character(round(P,2))),
              )
  eciLM <- round(exp(confint(model)), 2)
  res = cbind(paste0(round(exp(coef(model)),2)," [",eciLM[,1],", ",eciLM[,2],"]"), paste0(t[,4],t[,5])) %>%
        as.data.frame(.)
  colnames(res) <- c("ORCI","P")
  rownames(res) <- names(model$coeff)
  #Add spaces
  for (i in c(4,10,15,19,26)) {
    res = add_row(res, ORCI = " ", P = " ", .before = i)
    res = add_row(res, ORCI = "-", P = "-", .before = i + 1)
  }
  res <- res[-1,]
  rownames(res) <- NULL
  return(res)
}
res1 = LMRes(RTLM_Y1)
res2 = LMRes(RTLM_Y2)
res = cbind(res1,res2)
names(res) <- c("ORCI_Y1","P1","ORCI_Y2","P2")
kable(res,"html")

In [ ]:
%%R
res1 = LMRes(NCLM_Y1)
res2 = LMRes(NCLM_Y2)
res = cbind(res1,res2)

kable(res,"html")

In [ ]:
%%R
CPHRes <- function(model, spaces = TRUE, rows = FALSE) {
  t <- coef(summary(model))[,5] %>%
     as.data.frame(.)
  names(t) <- c("P")
  t <- t %>%
       mutate(ast = case_when(P < 0.001 ~ "***",
                         P < 0.01 ~ "**",
                         P < 0.05 ~ "*",
                         TRUE ~ ""),
              P = if_else(P<0.01, formatC(P, digits=2, format = "e"), as.character(round(P,2))),
              )
  eciLM <- round(exp(confint(model)), 2)
  res = cbind(paste0(round(exp(coef(model)),2)," [",eciLM[,1],", ",eciLM[,2],"]"), paste0(t[,1],t[,2])) %>%
        as.data.frame(.)
  colnames(res) <- c("ORCI","P")

  eciLM <- round(exp(confint(model)), 2)
  if (!rows) {
    rownames(res) <- NULL
  } else {
    rownames(res) <- names(model$coef)
  }
  #Add spaces
  if (spaces){
    for (i in c(3,9,14,18,25)) {
      res = add_row(res, ORCI = " ", P = " ", .before = i)
      res = add_row(res, ORCI = "-", P = "-", .before = i + 1)
    }
  }

  return(res)
}

res1 = CPHRes(LTF_Y1,rows=F)
res2 = CPHRes(LTF_Y2)
res = cbind(res1,res2)
kable(res,"html")


In [ ]:
%%R
res1 = CPHRes(UP2_Y1,spaces=F,rows=F)
res2 = CPHRes(UP2_Y2,spaces=F,rows=F)
res = cbind(res1,res2)
kable(res,"html")

### Different Timelines

In [ ]:
%%R
round(LTF$coef, 3)

In [ ]:
%%R
LTFdf_t <- mutate(LTFdf, BL_UP2 = if_else(BL_UP2 > 25, 0, 1)) #BL_AGE = round(BL_AGE, -1), BL_DDX = round(BL_DDX, -1), )
covars <- c("SEX", "RACE", "EDUC", "INCM", "RSRC", "EMPL", "BL_UP2", "BL_MCI", "IS_DEP")

for (i in 1:length(covars)){
  survltf <- eval(parse(text = paste("survfit(Surv(daysB, LTF == 1) ~ ",covars[i],", data = LTFdf_t)")))
  g <- ggsurvplot(fit = survltf,
      xlab = "Days Since Baseline",
      ylab = "Probability of Patient Remaining in Study",
      title = "Likelihood of Attrition in Fox Insight",
      legend.title = "",
      risk.table.y.text = F,
      risk.table = TRUE
      )

  print(g)
}

In [ ]:
%%R
#Loads dataset created in python
LTFdf = fread("FOX_TL2_LTF48.csv")

print("******************PRE COVID*****************")

LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (SEX + BL_AGE + RACE + EDUC + INCM + RSRC + EMPL + BL_DDX + BL_UP2 + BL_MCI + IS_DEP),
             data = LTFdf)
print(summary(LTF))

In [ ]:
%%R
#Loads dataset created in python
LTFdf = fread("FOX_TL3_LTF12.csv")

print("******************COVID FU*****************")

LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (SEX + BL_AGE + RACE + EDUC + INCM + RSRC + EMPL + BL_DDX + BL_UP2 + BL_MCI + IS_DEP),
             data = LTFdf)
print(summary(LTF))

In [ ]:
%%R
#Loads dataset created in python
LTFdf = fread("FOX_TL4_LTF12.csv")

print("******************COVID RECRUITMENT*****************")

LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (SEX + BL_AGE + RACE + EDUC + INCM + RSRC + EMPL + BL_DDX + BL_UP2 + BL_MCI + IS_DEP),
             data = LTFdf)
print(summary(LTF))

In [ ]:
%%R
#Loads dataset created in python
LTFdf = fread("FOX_TL5_LTF15.csv")

print("******************POST TIME WINDOW CHANGE*****************")

LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (SEX + BL_AGE + RACE + EDUC + INCM + RSRC + EMPL + BL_DDX + BL_UP2 + BL_MCI + IS_DEP),
             data = LTFdf)
print(summary(LTF))

## Create test dataset for attrition

In [ ]:
# According to Yasir and Josh, ReturnPD visits are assigned at each expected visit,
# We can use this to determine how long a participant was in the study

#Load return PD data
RPD = pd.read_csv(f"{datafolder}/FOX/ReturnPD.csv")

#Identify how old the participant would have been at the last expected visit
lastage = RPD.pivot_table(index="fox_insight_id", values = "age", aggfunc = "max")

#Age at enrollment is collected from Users data
U = (pd.read_csv(f"{datafolder}/FOX/Users.csv")
        .set_index("fox_insight_id")
        .rename({"AgeAtEnrollment":"BL_age"},axis=1)
        .loc[:,"BL_age"])

#Create follow-up time based on lastest age and enrollment age
FU = pd.concat([U, lastage],axis=1).dropna()
FU.loc[:,"FUTIME"] = ((FU.age - FU.BL_age) * 12).round(0)

#Estimate enrollment date based on date of data pull, follow-up time, and time window of 3 months
dts = [dt.datetime.today() - dt.timedelta(weeks = fu * 4) for fu in FU.FUTIME]

FU.loc[:,"earliest_enroll_dt"] = [f"{(date - dt.timedelta(weeks = 12)).month}/{date.year}" for date in dts]

FU.loc[:,"lastest_enroll_dt"] = [f"{dt.month}/{dt.year}" for dt in dts]

FU.loc[:,"est_enroll_dt"] = [date.date() for date in dts]

#Using data from GP2 harmonization, we can determine which participants have recently completed surveys
ACT = (pd.read_csv(f"{datafolder}/FOX/FOX_processed.csv")
        .pivot_table(index="participant_id", values = "visit_month", aggfunc = "max"))

FU = FU.join(ACT)

FU.loc[:,"INACT"] = FU.FUTIME - FU.visit_month

#Participants with INACT less than zero have changes to diagnosis status
print(f"{FU[FU.INACT < 0].index.nunique()} participants pruned due to negative inactive time")
FU = FU[FU.INACT > 0]

In [ ]:
print(f"{FU.index.nunique()} participants")
px.histogram(FU.est_enroll_dt)

In [ ]:
#What year-long enrollment period collects the most participants?
max = 0
for year in [2016,2017,2018,2019]:
  for month in range(1,13):
    n = ((dt.datetime(year=year,month=month,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=year+1, month=month,day = 1))).sum()
    if n > max:
      print(f"The new max enroll start date is {month}/{year} with {n} participants")
      max = n

In [ ]:
enroll_dts = pd.to_datetime(FU.est_enroll_dt)


tl1 = ((dt.datetime(year=2018,month=4,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=2019, month=4,day = 1)))
print(f'{tl1.sum()} participants in max enroll timeline')
tl2 = ((dt.datetime(year=2017,month=10,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=2018, month=10,day = 1)))
print(f'{tl2.sum()} participants in pre-COVID timeline')
tl3 = ((dt.datetime(year=2018,month=3,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=2019, month=3,day = 1)))
print(f'{tl3.sum()} participants in COVID follow-up timeline')
tl4 = ((dt.datetime(year=2019,month=3,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=2020, month=3,day = 1)))
print(f'{tl4.sum()} participants in COVID recruitment timeline')
tl5 = ((dt.datetime(year=2019,month=9,day=1) < enroll_dts) & (enroll_dts < dt.datetime(year=2020, month=9,day = 1)))
print(f'{tl5.sum()} participants in post time window change timeline')



In [ ]:
for thresh in range(3,24,3):
  print(f"{((FU.INACT > thresh).sum() / len(FU.INACT) * 100).round(2)} percent attrition with threshold of {thresh}")
px.histogram(FU.INACT)

In [ ]:
FU.loc[:,"ACT"] = [fu - ina if ina > 12 else np.nan for fu, ina in zip(FU.FUTIME,FU.INACT)]
FU.loc[:,"drop_dt"] = FU.est_enroll_dt + (FU.ACT * 4)

FU

In [ ]:
#Test create dataset for LTF analysis
# Set parameters
## Load FOX only once to reduce processing time
# FOX = pd.read_csv(f'{datafolder}/FOX/FOX_processed.csv')
## Set timeline of interest
tl = tl2[tl2].copy()
tln = "TL2"
FUP = 24

#Previous runs
# TL1/24
# TL5/15562

#Create dataframe with only timeline participants with two year follow-up
## Subset FOX dataset
FOX1 = FOX[FOX.participant_id.isin(tl.index)]

## Round visit_month down to visit window start
FOX1.insert(loc=1,column = "VM", value = (FOX1.visit_month // 3) * 3)
FOX1.insert(loc=1, column = "daysB", value = (FOX1.visit_month * 30))

## Drop visit_months greater than 24 months
FOX1 = FOX1[FOX1.VM <= FUP]

print(f"{FOX1.participant_id.nunique()} participants before cleaning")

FOX1 = FOX1.set_index("participant_id")

BL_cols = ["mds_updrs_part_ii_summary_score","gds15_total_score"]

for col in BL_cols:
  FOXt = FOX1[~FOX1.index.duplicated(keep="first")].loc[:,col]
  FOXt.name = f"BL_{col.upper().split('_')[0]}"
  FOX1 = FOX1.join(FOXt)

FOX2 = FOX1[~FOX1.index.duplicated(keep="last")]

#Create additional variables
FOX2.loc[:,"education_years"] = FOX2.education_level.map({'High School/GED':0, 'Some college without degree':0,
       'Professional or doctoral degree':1, "Master's degree":1, "Bachelor's degree":1,
       'Associate degree college':1, '<High School':0})

FOX2.loc[:,"BL_durdx"] = FOX2.age_at_baseline - FOX2.age_at_diagnosis

cov = ["daysB","visit_month","age_at_baseline","age_at_diagnosis","BL_durdx","sex","education_years","race","BL_MDS","BL_GDS15"]

for col in cov:
  print(f"{FOX2.loc[:,col].isna().sum()} participants removed due to {col}")
  FOX2 = FOX2.dropna(subset = [col])

FOX2 = FOX2.loc[:,cov]

print(f"{FOX2.index.nunique()} participants after cleaning")

FOX2.loc[:,"LTF"] = [1 if vm < FUP else 0 for vm in FOX2.visit_month]

FOX2.to_csv(f"FOX_LTF{FUP}_{tln}.csv",index=False)
FOX2

In [ ]:
%%R
#Loads dataset created in python
LTFdf = fread("FOX_LTF24_TL2.csv")

print("******************ST COXPH*****************")

LTF <- coxph(formula = Surv(visit_month, LTF == 1) ~ (sex + education_years + age_at_baseline + BL_durdx
                                                      + BL_MDS + BL_GDS15), data = LTFdf)
print(summary(LTF))

In [ ]:
%%R
#Loads dataset created in python
LTFdf = fread("FOX_LTF15_TL5.csv")

print("******************ST COXPH*****************")

LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (sex + education_years + age_at_baseline + BL_durdx
                                                      + BL_MDS + BL_GDS15), data = LTFdf)
print(summary(LTF))

In [ ]:
FOX = pd.read_csv(f'{datafolder}/FOX/FOX_processed.csv')
FOX.insert(loc=1, column = "previous_vm", value = FOX.groupby("participant_id").shift(1).loc[:,"visit_month"])
FOX.insert(loc=1, column = "SOA", value = [(vm // 3) * 3 for vm in FOX.visit_month])
FOX = FOX.drop_duplicates(subset = ["participant_id","SOA"])
#.pivot(index="participant_id", columns = "visit_month")
px.histogram(FOX.visit_month, x = "visit_month")

In [ ]:
FOX_count = FOX.pivot_table(index="participant_id",values = "visit_month",aggfunc="count").rename({"visit_month":"visit_count"}, axis=1)
px.histogram(FOX_count, x = "visit_count")

In [ ]:
FOX_lastvm = (FOX.drop_duplicates(subset = "participant_id",keep="last")
                 .loc[:,["participant_id","SOA"]]
                 .rename({"SOA":"LastVM"},axis=1)
                 .set_index("participant_id"))

FOX_freq = FOX_count.join(FOX_lastvm)

FOX_freq.loc[:,"freq"] = (FOX_freq.LastVM / FOX_freq.visit_count).round(1)

px.histogram(FOX_freq, x="freq")

In [ ]:
FOX2 = FOX_lastvm[FOX_lastvm.LastVM >= 24].index
FOX2_df = FOX[(FOX.participant_id.isin(FOX2)) & (FOX.SOA <= 24)]
px.histogram(FOX2_df.pivot_table(index="participant_id", values = "SOA", aggfunc = "count") / .09, x= "SOA", labels = {"SOA":"Compliance Rate"})
#Expected visits [BL, 3, 6, 9, 12, 15, 18, 21, 24] = 9

In [ ]:
FOX2_df.iloc[0:50,51:100]

In [ ]:
FOX = pd.read_csv(f'{datafolder}/FOX/FOX_processed.csv')
FOX_UP2 = FOX.pivot_table(index="participant_id",values = "mds_updrs_part_ii_summary_score", aggfunc="count")
FOX_UP2.columns = ["UP2"]

px.histogram(FOX_UP2)

In [ ]:
U = pd.read_csv(f'{datafolder}/FOX/Users.csv')
U_cols = {"fox_insight_id":"ID",
              "AgeAtEnrollment":"BL_AGE",
              "days_elapsed":"BL_DATE",
              "InitPDDiagAge":"DX_AGE",
             }

U = U.rename(U_cols, axis=1).loc[:,U_cols.values()]

U.loc[:,"BL_durdx"] = U.BL_AGE - U.DX_AGE

BMS_cols =  {"fox_insight_id":"ID",
              "AgeAtEnrollment":"BL_AGE",
              "days_elapsed":"BL_DATE",
              "InitPDDiagAge":"DX_AGE",
             }

BMS = pd.read_csv(f'{datafolder}/FOX/Brief_Motor_Screen.csv')
BMS.sort_values(["fox_insight_id","days_elapsed"]).head(50)

# PPMI

In [ ]:
#Loads harmonized PPMI data from GP2
PPMI = pd.read_csv(f'{datafolder}/PPMI_GP2.csv')

#Loads info on in or out status
PS = pd.read_csv(f'{datafolder}/Participant_Status.csv')
PS.loc[:,"ENROLL_STATUS"] = PS.ENROLL_STATUS.str.lower()

PS = PS.loc[PS.ENROLL_STATUS.isin(["enrolled","withdrew","declined","complete"])]

PS.loc[:,"ENROLL_DATE"] = pd.to_datetime(PS.ENROLL_DATE)

#Create "Loss to Follow-Up" Variable, then subset data
PS.loc[:,"censure_date"] = (pd.to_datetime(PS.STATUS_DATE) - pd.Timestamp("1970-01-01")) // pd.Timedelta("1D")

PS.loc[:,"days_to_cens"] = (pd.to_datetime(PS.STATUS_DATE) - pd.to_datetime(PS.ENROLL_DATE)) // pd.Timedelta("1D")
PS.loc[:,"LTF2"] = [1 if ((s in ["withdrew","declined"]) & (d <= 730)) else 0 for s,d in zip(PS.ENROLL_STATUS,PS.days_to_cens)]
PS.loc[:,"LTF5"] = [1 if ((s in ["withdrew","declined"]) & (d <= 1825)) else 0 for s,d in zip(PS.ENROLL_STATUS,PS.days_to_cens)]

PS = PS.loc[:,["PATNO","COHORT","censure_date","LTF2","LTF5"]]

PPMI = PPMI.merge(PS, left_on =  "participant_id", right_on = "PATNO")

PPMI = PPMI[PPMI.Phenotype.isin(["Control","PD"])]

PPMI.loc[:,"ethrac"] = ["HISP" if e == "Hispanic or Latino"
                      else "ASIA" if r == "Asian"
                      else "BLAC" if r == "Black or African American"
                      else "WHIT" if r == "White"
                      else "OTH" if r in ["Multi-racial","Other","American Indian or Alaska Native"]
                      else np.nan for e,r in zip(PPMI.ethnicity,PPMI.race)]

PPMI.loc[:,"educlvl"] = ["<HS" if yr < 12
                         else "HS" if yr < 14
                         else "UNI" if yr < 18
                         else "UNI+" if yr >= 18
                         else np.nan for yr in PPMI.education_years]

PPMI.loc[:,"cog3"] = ["DEM" if mca < 18
                      else "MCI" if mca < 26
                      else "NORM" for mca in PPMI.moca_total_score]

for LTF in ["LTF2","LTF5"]:
  PPMI_cols = {"ID":"participant_id",
              "BL_DATE":'date_baseline',
              "DATE":'date_visit',
              "VM":'visit_month',
              "CENS_DT":"censure_date",
              "LTF":LTF,
              "PHENO":'Phenotype',
              'SEX':'sex',
              'RACE':"ethrac",
              "ETHN":"ethnicity",
              'STUDY_ARM':'study_arm',
              'EDUC':'educlvl',
              "DX_AGE":'age_at_diagnosis',
              'BL_AGE':'age_at_baseline',

              "UP1_COG":"code_upd2101_cognitive_impairment",
              "UP1_PSYC":"code_upd2102_hallucinations_and_psychosis",
              "UP1_DEP":"code_upd2103_depressed_mood",
              "UP1_ANX":"code_upd2104_anxious_mood",
              "UP1_APAT":"code_upd2105_apathy",

              "UP2_SPEEC": "code_upd2201_speech",
              "UP2_SALIV":"code_upd2202_saliva_and_drooling",
              "UP2_SWALL":"code_upd2203_chewing_and_swallowing",
              "UP2_EAT":"code_upd2204_eating_tasks",
              "UP2_DRESS":"code_upd2205_dressing",
              "UP2_HYGEI":"code_upd2206_hygiene",
              "UP2_HNDWR":"code_upd2207_handwriting",
              "UP2_HOBBY":"code_upd2208_doing_hobbies_and_other_activities",
              "UP2_BED":"code_upd2209_turning_in_bed",
              "UP2_TREM":"code_upd2210_tremor",
              "UP2_GETUP":"code_upd2211_get_out_of_bed_car_or_deep_chair",
              "UP2_WALK":"code_upd2212_walking_and_balance",
              "UP2_FREEZ":"code_upd2213_freezing",
              "UP2":'mds_updrs_part_ii_summary_score',
              "UP3":'mds_updrs_part_iii_summary_score',
              "APAT":"code_upd2105_apathy",
              "COG":'moca_total_score',
              'COG3':'cog3',
              "DEP":'depress_test_score',
              "IS_DEP":'depress_test_score/5',
  }


  PPMI_CS, PPMI_LNG = PrepKM(PPMI, PPMI_cols)

  # PPMI_CRA = PPMI_CS.copy()
  # PPMI_CRA.loc[:,"CRA_MCI"] = [1 if mca < 26 else 2 if ltf == 1 else 0 for mca,ltf in zip(PPMI_CS.COG,PPMI_CS.LTF)]
  # PPMI_CRA.loc[:,"CRA_DEM"] = [1 if mca < 18 else 2 if ltf == 1 else 0 for mca,ltf in zip(PPMI_CS.COG,PPMI_CS.LTF)]
  # PPMI_CRA.loc[:,"CRA_DEP"] = [1 if dep == 1 else 2 if ltf == 1 else 0 for dep,ltf in zip(PPMI_CS.DEP,PPMI_CS.LTF)]

  PPMI_CS.to_csv(f'PPMI_{LTF}_CS.csv', index=False)
  PPMI_LNG.to_csv(f'PPMI_{LTF}_LNG.csv', index=False)

## Two Year LTF Analysis

In [ ]:
%%R
PPMI_LTF2_CS <- fread("PPMI_LTF2_CS.csv")
# TI_LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (sex + education_years + BL_AGE + BL_UP2 + BL_UP3 + BL_MCA + BL_GDS), data = filter(kd, Phenotype == "PD"))
# TI_LTF <- coxph(formula = Surv(TSTART, daysB, LTF_TD == 1) ~ (EDUC + SEX + BL_AGE + BL_durdx + UP2 + UP3 + I(COG < 27) + I(DEP>4)), data = filter(PPMI_LNG, PHENO == "PD"))
# print(summary(TI_LTF))

PPMI2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (SEX + BL_AGE + EDUC + BL_durdx + BL_UP2 + BL_UP3 + I(BL_COG < 26) + I(BL_DEP>4)), data = filter(PPMI_LTF2_CS, PHENO == "PD"))
print(summary(PPMI2))

PPMI2M <- coxph(formula = Surv(daysB, LTF == 1) ~ (BL_UP2_SPEEC + BL_UP2_SALIV +
        BL_UP2_SWALL + BL_UP2_EAT + BL_UP2_DRESS + BL_UP2_HYGEI + BL_UP2_HNDWR + BL_UP2_HOBBY + BL_UP2_BED
        + BL_UP2_TREM + BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_FREEZ), data = filter(PPMI_LTF2_CS, PHENO == "PD"))
print(summary(PPMI2M))


In [ ]:
%%R
CPHRes <- function(model, spaces = TRUE, rows = FALSE) {
  t <- coef(summary(model))[,5] %>%
     as.data.frame(.)
  names(t) <- c("P")
  t <- t %>%
       mutate(ast = case_when(P < 0.001 ~ "***",
                         P < 0.01 ~ "**",
                         P < 0.05 ~ "*",
                         TRUE ~ ""),
              P = if_else(P<0.01, formatC(P, digits=2, format = "e"), as.character(round(P,2))),
              )
  eciLM <- round(exp(confint(model)), 2)
  res = cbind(paste0(round(exp(coef(model)),2)," [",eciLM[,1],", ",eciLM[,2],"]"), paste0(t[,1],t[,2])) %>%
        as.data.frame(.)
  colnames(res) <- c("ORCI","P")

  eciLM <- round(exp(confint(model)), 2)
  #Add spaces
  if (spaces){
    for (i in c(3,9,14,22)) {
      res = add_row(res, ORCI = " ", P = " ", .before = i)
      res = add_row(res, ORCI = "-", P = "-", .before = i + 1)
    }
  }
  if (!rows) {
    rownames(res) <- NULL
  }
  return(res)
}

res <- CPHRes(PPMI2M, spaces=FALSE)
kable(res,"html")

## Five Year LTF Analysis

In [ ]:
%%R
PPMI_LTF5_CS <- fread("PPMI_LTF5_CS.csv")
# TI_LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (sex + education_years + BL_AGE + BL_UP2 + BL_UP3 + BL_MCA + BL_GDS), data = filter(kd, Phenotype == "PD"))
# TI_LTF <- coxph(formula = Surv(TSTART, daysB, LTF_TD == 1) ~ (EDUC + SEX + BL_AGE + BL_durdx + UP2 + UP3 + I(COG < 27) + I(DEP>4)), data = filter(PPMI_LNG, PHENO == "PD"))
# print(summary(TI_LTF))

PPMI5 <- coxph(formula = Surv(daysB, LTF == 1) ~ (EDUC + SEX + BL_AGE + BL_durdx + BL_UP2 + BL_UP3 + I(BL_COG < 27) + I(BL_DEP>4)), data = filter(PPMI_LTF5_CS, PHENO == "PD"))
print(summary(PPMI5))




## Item-Level UP2 Analysis

In [ ]:
%%R
PPMI2UP2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (BL_UP2_SPEEC + BL_UP2_SALIV +
        BL_UP2_SWALL + BL_UP2_EAT + BL_UP2_DRESS + BL_UP2_HYGEI + BL_UP2_HNDWR + BL_UP2_HOBBY + BL_UP2_BED
        + BL_UP2_TREM + BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_FREEZ), data = filter(PPMI_LTF2_CS, PHENO == "PD"))
print(summary(PPMI2UP2))

PPMI5UP2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (BL_UP2_SPEEC + BL_UP2_SALIV +
        BL_UP2_SWALL + BL_UP2_EAT + BL_UP2_DRESS + BL_UP2_HYGEI + BL_UP2_HNDWR + BL_UP2_HOBBY + BL_UP2_BED
        + BL_UP2_TREM + BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_FREEZ), data = filter(PPMI_LTF5_CS, PHENO == "PD"))
print(summary(PPMI5UP2))

In [ ]:
%%R
kable(CPHRes(PPMI2UP2,spaces=F))

## Mobility Measure Test

In [ ]:
%%R

PPMI_LTF2_CS <- mutate(PPMI_LTF2_CS, BL_UP2_MOB = BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_DRESS)
PPMI_LTF5_CS <- mutate(PPMI_LTF5_CS, BL_UP2_MOB = BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_DRESS)

PPMI2M <- coxph(formula = Surv(daysB, LTF == 1) ~ (EDUC + SEX + BL_AGE + BL_durdx + BL_UP2_MOB + BL_UP3 + I(BL_COG < 27) + I(BL_DEP>4)), data = filter(PPMI_LTF2_CS, PHENO == "PD"))
print(summary(PPMI2M))

PPMI5M <- coxph(formula = Surv(daysB, LTF == 1) ~ (EDUC + SEX + BL_AGE + BL_durdx + BL_UP2_MOB + BL_UP3 + I(BL_COG < 27) + I(BL_DEP>4)), data = filter(PPMI_LTF5_CS, PHENO == "PD"))
print(summary(PPMI5M))

# NET-PD-LS1 Analysis

In [ ]:
LS1 = pd.read_csv(f'{datafolder}/LS1_processed.csv')
LS1d = pd.read_csv(f'{datafolder}/LS1_demo.csv').loc[:,["SUBJID","occmosty","occcurry"]]
LS1s = pd.read_csv(f'{datafolder}/LS1_status.csv').loc[:,["SUBJID","term","days_term"]].merge(LS1d, how = "left")
LS1s.loc[:,"term"] = LS1s.term.fillna(0)

LS1 = LS1.merge(LS1s, how = "left", left_on = "participant_id", right_on  = "SUBJID")

LS1.loc[:,"mds_updrs_part_ii_summary_score"] = ((LS1.updrs_part_ii_summary_score * 1.1) + 0.2)
LS1.loc[:,"mds_updrs_part_iii_summary_score"] = ((LS1.updrs_part_iii_summary_score * 1.2) + 2.3)

# LS1_LTF = LS1.pivot_table(index="participant_id", values = "visit_month", aggfunc = "max")
# LS1_LTF.loc[:,"LTF_TI"] = [0 if i >= 60 else 1 for i in LS1_LTF.iloc[:,0]]
# LS1_LTF.columns = ["LTF_VM","LTF"]

# LS1_LTF = LS1_LTF.reset_index()

# LS1 = LS1.merge(LS1_LTF)

LS1.loc[:,"education_years"] = LS1.education_level.map({'High School/GED':0, 'Some college without degree':0,
       'Professional or doctoral degree':1, "Bachelor's degree":1,
       'Associate degree college':1, '<High School':0})
LS1 = LS1.rename({"term":"LTF5"},axis=1)
LS1.loc[:,"LTF2"] = [1 if ((t==1) & (dt >= 730)) else  0 for t,dt in zip(LS1.LTF5, LS1.days_term)]
LS1.loc[:,"LTF5"] = LS1.LTF5.mask(LS1.LTF2 == 1, np.nan)

for LTF in ["LTF2","LTF5"]:
  ls1_dict = {"ID":'participant_id',
              "BL_DATE":'date_baseline',
              "DATE":'date_visit',
              "VM":'visit_month',
              "CENS_DT":"days_term",
              "LTF":LTF,
              "PHENO":'Phenotype',
              "SEX":'sex',
              "EDUC":"education_years",
              "RACE":"race",
              "OCCMOST":"occmosty",
              "ARM":'study_arm',
              "DX_AGE":'age_at_diagnosis',
              "BL_AGE":'age_at_baseline',

              "UP1_COG":"code_upd101_intellectual_impairment",
              "UP1_PSYC":"code_upd102_thought_disorder",
              "UP1_DEP":"code_upd103_depression",
              "UP1_APAT":"code_upd104_motivation",

              "UP2_SPEEC": "code_upd105_speech",
              "UP2_SALIV":"code_upd106_salivation",
              "UP2_SWALL":"code_upd107_swallowing",
              "UP2_EAT":"code_upd109_eating_tasks",
              "UP2_DRESS":"code_upd110_dressing",
              "UP2_HYGEI":"code_upd111_hygiene",
              "UP2_HNDWR":"code_upd108_handwriting",
              "UP2_BED":"code_upd112_bed",
              "UP2_TREM":"code_upd116_tremor",
              "UP2_GETUP":"code_upd127_arising_from_chair",
              "UP2_WALK":"code_upd115_walking",
              "UP2_FREEZ":"code_upd114_freezing_of_gait",
              "UP2_FALLS":"code_upd113_falling",
              "UP2_SENS":"code_upd117_sensory_complaints",


              "UP2":'mds_updrs_part_ii_summary_score',
              "UP3":'mds_updrs_part_iii_summary_score',
              "COG":'scopa_cog_total_score',
              "DEP":'depress_test_score',
              "IS_DEP":"depress_test_score/15"}

  LS1_CS, LS1_LNG = PrepKM(LS1, ls1_dict)

  # LS1_LNG.loc[:,"LTF_TD"] = [0 if k == 0 else 1 if i > j else 0 for i, j, k in zip(LS1_LNG.daysB, LS1_LNG.CENS_DT, LS1_LNG.LTF)]

  LS1_CS.to_csv(f'LS1_{LTF}_CS.csv', index=False)
  LS1_LNG.to_csv(f'LS1_{LTF}_LNG.csv', index=False)
  LS1_LNG.head(50)

In [ ]:
%%R
##Loads libraries required for analysis
pack <- "/content/gdrive/My Drive/R/packages"

library(data.table, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library(dplyr, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library(lubridate, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library("ggpubr", lib.loc = pack)
library("ggplot2", lib.loc = pack)
library("ggpubr", lib.loc = pack)
library("survival", lib.loc = pack)
library("survminer", lib.loc = pack)
library(tidyr, lib.loc = pack)

#Loads dataset created in python
LS1_LTF2_CS = fread("LS1_LTF2_CS.csv")
LS1_LTF5_CS = fread("LS1_LTF5_CS.csv")



In [ ]:
%%R
# TI_LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (sex + education_years + BL_AGE + BL_UP2 + BL_UP3 + BL_MCA + BL_GDS), data = filter(kd, Phenotype == "PD"))
LS12 <- coxph(formula = Surv(daysB, LTF == 1) ~ (EDUC + SEX  + BL_AGE + BL_durdx +BL_UP2 + BL_UP3 + I(BL_COG<24) + I(BL_DEP>13)), data = LS1_LTF2_CS)
print(summary(LS12))

LS15 <- coxph(formula = Surv(daysB, LTF == 1) ~ (EDUC + SEX  + BL_AGE + BL_durdx + +BL_UP2 + BL_UP3 + BL_COG + BL_DEP), data = LS1_LTF5_CS)
print(summary(LS15))

## Item-Level UP2 Analysis

In [ ]:
%%R
# TI_LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (sex + education_years + BL_AGE + BL_UP2 + BL_UP3 + BL_MCA + BL_GDS), data = filter(kd, Phenotype == "PD"))
TI_LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (BL_UP2_SPEEC + BL_UP2_SALIV +
        BL_UP2_SWALL + BL_UP2_EAT + BL_UP2_DRESS + BL_UP2_HYGEI + BL_UP2_HNDWR + BL_UP2_BED
        + BL_UP2_TREM + BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_FREEZ), data = LS1_LTF2_CS)
print(summary(TI_LTF))

TI_LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (BL_UP2_SPEEC + BL_UP2_SALIV +
        BL_UP2_SWALL + BL_UP2_EAT + BL_UP2_DRESS + BL_UP2_HYGEI + BL_UP2_HNDWR + BL_UP2_BED
        + BL_UP2_TREM + BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_FREEZ), data = LS1_LTF5_CS)
print(summary(TI_LTF))

In [ ]:
%%R
LS1_LTF2_CS <- mutate(LS1_LTF2_CS, BL_UP2_MOB = BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_DRESS)
LS1_LTF5_CS <- mutate(LS1_LTF5_CS, BL_UP2_MOB = BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_DRESS)

TI_LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (EDUC + SEX  + BL_AGE + BL_durdx + +BL_UP2_MOB + BL_UP3 + I(BL_COG<24) + I(BL_DEP>13)), data = LS1_LTF2_CS)
print(summary(TI_LTF))

TI_LTF <- coxph(formula = Surv(daysB, LTF == 1) ~ (EDUC + SEX  + BL_AGE + BL_durdx + +BL_UP2_MOB + BL_UP3 + I(BL_COG<24) + I(BL_DEP>13)), data = LS1_LTF5_CS)
print(summary(TI_LTF))

In [ ]:
import tableone as tab1

pdat = LS1_LNG.drop_duplicates(keep="first", subset = "ID")
pdat = pdat.merge(LS1_LNG.drop_duplicates(subset = "ID", keep = "last").loc[:,["ID","daysB"]].rename({"daysB":"FUTIME"},axis=1), how = "left")
pdat = pdat[pdat.FUTIME > 0]
pdat.loc[:,"FUTIME"] = (pdat.FUTIME / 365.25).round(1)
pdat = pdat[pdat.PHENO == "PD"]
pdat.loc[:,"IS_COG"] = ["DEM" if c <= 17 else "MCI" if c <= 24 else "NORM" for c in pdat.COG]
pdat.loc[:,"IS_DEP"] = ["DEP" if c >= 13 else "NORM" for c in pdat.DEP]

tab1.TableOne(pdat, columns = ["BL_AGE","durdx","FUTIME", "RACE", "SEX", "EDUC", "UP2", "UP3", "IS_COG", "IS_DEP"],
              nonnormal = ["durdx","FUTIME","UP2","UP3"], groupby = ["LTF"])

# SURE & STEADY

In [ ]:
SUST  = pd.read_csv(f"{datafolder}/PDBP_processed.csv")
SUST = SUST[SUST.study_arm.isin(["245_PD","247_PD"])]

SUST_LTF = SUST.drop_duplicates(subset = "participant_id", keep = "last").loc[:,["participant_id","study_arm","visit_month"]]

SUST_LTF.loc[:,"lost_to_follow_up"] = [1 if (sa == "247_PD") & (vm < 24)
                          else 1 if (sa == "245_PD") & (vm < 24)
                          else 0 for sa, vm in zip(SUST_LTF.study_arm, SUST_LTF.visit_month)]
SUST_LTF.drop(["visit_month","study_arm"], axis=1, inplace = True)

SUST = SUST.merge(SUST_LTF)

SUST.loc[:,"BL_durdx"] = SUST.age_at_baseline - SUST.age_at_diagnosis

SUST.loc[:,"education_years"] = SUST.education_level.map({'High School/GED':0, 'Some college without degree':0,
       'Professional or doctoral degree':1, "Bachelor's degree":1,
       'Associate degree college':1, '<High School':0})

gp2_cols =  ["participant_id", "sex", "lost_to_follow_up", "education_level", "moca_total_score","depress_test_score",
             "code_upd2201_speech","code_upd2202_saliva_and_drooling","code_upd2203_chewing_and_swallowing",
            "code_upd2204_eating_tasks","code_upd2205_dressing","code_upd2206_hygiene",
            "code_upd2207_handwriting","code_upd2208_doing_hobbies_and_other_activities",
            "code_upd2209_turning_in_bed","code_upd2210_tremor","code_upd2211_get_out_of_bed_car_or_deep_chair",
            "code_upd2212_walking_and_balance","code_upd2213_freezing","code_upd2105_apathy",
            "mds_updrs_part_ii_summary_score","mds_updrs_part_iii_summary_score"]

SUST_cols = {"ID":"participant_id",
            "LTF":"lost_to_follow_up",
            'SEX':'sex',
            'STUDY_ARM':"study_arm",
            'EDUC':'education_years',
            "DX_AGE":"age_at_diagnosis",
            "DATE":"date_visit",
            "BL_DATE":"date_baseline",
            "BL_durdx":"BL_durdx",
            'BL_AGE':'age_at_baseline',

            "UP2":"mds_updrs_part_ii_summary_score",
            "UP3":"mds_updrs_part_iii_summary_score",

            "UP2_SPEEC": "code_upd2201_speech",
            "UP2_SALIV":"code_upd2202_saliva_and_drooling",
            "UP2_SWALL":"code_upd2203_chewing_and_swallowing",
            "UP2_EAT":"code_upd2204_eating_tasks",
            "UP2_DRESS":"code_upd2205_dressing",
            "UP2_HYGEI":"code_upd2206_hygiene",
            "UP2_HNDWR":"code_upd2207_handwriting",
            "UP2_HOBBY":"code_upd2208_doing_hobbies_and_other_activities",
            "UP2_BED":"code_upd2209_turning_in_bed",
            "UP2_TREM":"code_upd2210_tremor",
            "UP2_GETUP":"code_upd2211_get_out_of_bed_car_or_deep_chair",
            "UP2_WALK":"code_upd2212_walking_and_balance",
            "UP2_FREEZ":"code_upd2213_freezing",
            "UP2":'mds_updrs_part_ii_summary_score',
            "UP3":'mds_updrs_part_iii_summary_score',
            "APAT":"code_upd2105_apathy",
            "COG":'moca_total_score',
            "DEP":'code_upd2103_depressed_mood'
}

SUST_CS, SUST_LNG = PrepKM(SUST, SUST_cols)

# PPMI_CRA = PPMI_CS.copy()
# PPMI_CRA.loc[:,"CRA_MCI"] = [1 if mca < 26 else 2 if ltf == 1 else 0 for mca,ltf in zip(PPMI_CS.COG,PPMI_CS.LTF)]
# PPMI_CRA.loc[:,"CRA_DEM"] = [1 if mca < 18 else 2 if ltf == 1 else 0 for mca,ltf in zip(PPMI_CS.COG,PPMI_CS.LTF)]
# PPMI_CRA.loc[:,"CRA_DEP"] = [1 if dep == 1 else 2 if ltf == 1 else 0 for dep,ltf in zip(PPMI_CS.DEP,PPMI_CS.LTF)]

SUST_CS.to_csv('SUST_KM_CS.csv', index=False)
SUST_LNG.to_csv('SUST_KM_LNG.csv', index=False)
SUST_CS

## Item-Level UP2 Analysis

In [ ]:
SUST_CS[SUST_CS.STUDY_ARM == "245_PD"]

In [ ]:
%%R
##Loads libraries required for analysis
pack <- "/content/gdrive/My Drive/R/packages"

library(data.table, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library(dplyr, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library(lubridate, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library("ggpubr", lib.loc = pack)
library("ggplot2", lib.loc = pack)
library("ggpubr", lib.loc = pack)
library("survival", lib.loc = pack)
library("survminer", lib.loc = pack)
library(tidyr, lib.loc = pack)

#Loads dataset created in python
SUST_KM_CS = fread("SUST_KM_CS.csv")

print("******************ST COXPH*****************")
ST2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (SEX  + BL_AGE + +BL_UP2 + BL_UP3 + BL_COG + BL_DEP), data = filter(SUST_KM_CS, STUDY_ARM == "245_PD"))
print(summary(SUST2))

print("******************SU COXPH*****************")
SU2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (EDUC + SEX  + BL_AGE + BL_durdx + +BL_UP2 + BL_UP3 + BL_COG + BL_DEP), data = filter(SUST_KM_CS, STUDY_ARM == "247_PD"))
print(summary(SUST2))

# print("******************SUST COXPH UP2*****************")
# SUST2UP2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (BL_UP2_SPEEC + BL_UP2_SALIV +
#         BL_UP2_SWALL + BL_UP2_EAT + BL_UP2_DRESS + BL_UP2_HYGEI + BL_UP2_HNDWR + BL_UP2_HOBBY + BL_UP2_BED
#         + BL_UP2_TREM + BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_FREEZ), data = SUST_KM_CS)
# print(summary(SUST2UP2))


# SURE & STEADY Redux

In [ ]:
SUST  = pd.read_csv(f"{datafolder}/PPSUST_COMPLETE.csv")
SUST = SUST[SUST.study.isin(["SURE","STED"])]

SUST_LTF = SUST.drop_duplicates(subset = "AMPPD_ID", keep = "last").loc[:,["AMPPD_ID","study","days_in_study"]]

SUST_LTF.loc[:,"lost_to_follow_up"] = [1 if (sa == "SURE") & (vm < (365.25 * 2))
                          else 1 if (sa == "STED") & (vm < (365.25 * 2))
                          else 0 for sa, vm in zip(SUST_LTF.study, SUST_LTF.days_in_study)]
SUST_LTF.drop(["days_in_study","study"], axis=1, inplace = True)

SUST = SUST.merge(SUST_LTF)

SUST.loc[:,"DX_AGE"] = SUST.age_cnst_rnd - SUST.yrs_dx_base

SUST_cols = {"ID":"AMPPD_ID",
            "LTF":"lost_to_follow_up",
            'SEX':'gender_mf',
            'STUDY_ARM':"study",
            'EDUC':'EDUCYRS',
            "BL_durdx":"yrs_dx_base",
            "daysB":"study_dy",
            'BL_AGE':'age_cnst_rnd',
            'DX_AGE':"DX_AGE",
            'RACE':"RACE",
            "UP2":"UPDRS2",
            "UP3":"UPDRS3",
            "NHY":"NHY",

            "APAT":"NP1APAT",
            "COG":'NP1COG',
            "DEP":'NP1DPRS'
}

SUST_CS, SUST_LNG = PrepKM(SUST, SUST_cols)

# PPMI_CRA = PPMI_CS.copy()
# PPMI_CRA.loc[:,"CRA_MCI"] = [1 if mca < 26 else 2 if ltf == 1 else 0 for mca,ltf in zip(PPMI_CS.COG,PPMI_CS.LTF)]
# PPMI_CRA.loc[:,"CRA_DEM"] = [1 if mca < 18 else 2 if ltf == 1 else 0 for mca,ltf in zip(PPMI_CS.COG,PPMI_CS.LTF)]
# PPMI_CRA.loc[:,"CRA_DEP"] = [1 if dep == 1 else 2 if ltf == 1 else 0 for dep,ltf in zip(PPMI_CS.DEP,PPMI_CS.LTF)]

SUST_CS.to_csv('SUST_KM_CS.csv', index=False)
SUST_LNG.to_csv('SUST_KM_LNG.csv', index=False)
SUST_CS

In [ ]:
%%R
##Loads libraries required for analysis
pack <- "/content/gdrive/My Drive/R/packages"

library(data.table, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library(dplyr, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library(lubridate, lib.loc = pack, quietly = TRUE, verbose = FALSE)
library("ggpubr", lib.loc = pack)
library("ggplot2", lib.loc = pack)
library("ggpubr", lib.loc = pack)
library("survival", lib.loc = pack)
library("survminer", lib.loc = pack)
library(tidyr, lib.loc = pack)

#Loads dataset created in python
SUST_KM_CS = fread("SUST_KM_CS.csv")

print("******************ST COXPH*****************")
ST2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (EDUC + SEX + RACE + BL_AGE + +BL_UP2 + BL_UP3 + BL_COG + BL_DEP), data = filter(SUST_KM_CS, STUDY_ARM == "SURE"))
print(summary(ST2))

print("******************SU COXPH*****************")
SU2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (EDUC + SEX  + BL_AGE + BL_durdx + +BL_UP2 + BL_UP3 + BL_COG + BL_DEP), data = filter(SUST_KM_CS, STUDY_ARM == "STED"))
print(summary(SU2))

# print("******************SUST COXPH UP2*****************")
# SUST2UP2 <- coxph(formula = Surv(daysB, LTF == 1) ~ (BL_UP2_SPEEC + BL_UP2_SALIV +
#         BL_UP2_SWALL + BL_UP2_EAT + BL_UP2_DRESS + BL_UP2_HYGEI + BL_UP2_HNDWR + BL_UP2_HOBBY + BL_UP2_BED
#         + BL_UP2_TREM + BL_UP2_GETUP + BL_UP2_WALK + BL_UP2_FREEZ), data = SUST_KM_CS)
# print(summary(SUST2UP2))


#RCT Meta-Analysis

In [ ]:
%%R
results_rct[[1]]

In [ ]:
%%R
pack <- "/content/gdrive/My Drive/R/packages"
library(metafor, lib.loc = pack)

LS1 <- c(summary(LS12)$coef[,1],summary(LS12)$coef[,3])
SUST <- c(summary(SUST2)$coef[,1],summary(SUST2)$coef[,3])

results_rct <- data.frame(rbind(LS1, SUST))
num_vars <- (length(names(results_rct)) / 2)
for (i in 1:(num_vars-1)){
    model2 = rma(results_rct[[i]], sei=results_rct[[i + num_vars]], slab=c("NET-PD","SUST"), method='REML')

    print(names(results_rct)[i])
    print(model2)
    print("HR")
    print(exp(model2$b))
    print(exp(model2$ci.lb))
    print(exp(model2$ci.ub))

}

# All PDBP

In [ ]:
#We tried with just STEADY, but the lack of events is concerning to the overall power, so we try with all PDBP
GP2 = pd.read_csv(f"{datafolder}/PDBP_processed.csv")
GP2 = GP2[GP2.Phenotype == "PD"] #Keep only PD cases
# GP2 = GP2[~GP2.study_arm.isin(["204_PD","214_PD","231_PD","232_PD","235_PD","236_PD","247_PD"])] #Remove single visit studies
# GP2
GP2.drop_duplicates(subset = "participant_id",keep="last").pivot_table(index="study_arm",values = "visit_month", aggfunc = ["count","max"])



In [ ]:


gp2_cols =  ["participant_id", "moca_total_score","depress_test_score",
             "code_upd2201_speech","code_upd2202_saliva_and_drooling","code_upd2203_chewing_and_swallowing",
            "code_upd2204_eating_tasks","code_upd2205_dressing","code_upd2206_hygiene",
            "code_upd2207_handwriting","code_upd2208_doing_hobbies_and_other_activities",
            "code_upd2209_turning_in_bed","code_upd2210_tremor","code_upd2211_get_out_of_bed_car_or_deep_chair",
            "code_upd2212_walking_and_balance","code_upd2213_freezing","code_upd2105_apathy",
            "mds_updrs_part_ii_summary_score","mds_updrs_part_iii_summary_score"]

STED_GP2 = STED_GP2.loc[STED_GP2.moca_total_score.notna(),gp2_cols].drop_duplicates(subset = "participant_id", keep = "first")

STED = STED.merge(STED_GP2, left_on = "GUID", right_on = "participant_id").drop("participant_id",axis=1)

STED_cols = {"ID":"GUID",
            "daysB":"study_dy",
            "CENS_DT":"days_in_study_meds",
            "LTF":"lost_to_follow_up",
            'SEX':'gender_mf',
            'RACE':"RACE",
            'EDUC':'EDUCYRS',
            "DX_AGE":"DX_AGE",
            "BL_durdx":"BL_durdx",
            'BL_AGE':'age_cnst_rnd',


            "COG":"moca_total_score",

            "NUP1_COG":"NP1COG",
            "NUP1_PSYC":"NP1HALL",
            "NUP1_DEP":"NP1DPRS",
            "NUP1_ANX":"NP1ANXS",
            "NUP1_APAT":"NP1APAT",

            "UP1_DEP":"P1DEPRSN",
            "UP1_APAT":"P1MOTIVN",

            "UP2":"UPDRS2",
            "UP3":"UPDRS3",
            "OUP2":"OLD_UPDRS2",
            "OUP3":"OLD_UPDRS3",

            "UP2_SPEEC": "code_upd2201_speech",
            "UP2_SALIV":"code_upd2202_saliva_and_drooling",
            "UP2_SWALL":"code_upd2203_chewing_and_swallowing",
            "UP2_EAT":"code_upd2204_eating_tasks",
            "UP2_DRESS":"code_upd2205_dressing",
            "UP2_HYGEI":"code_upd2206_hygiene",
            "UP2_HNDWR":"code_upd2207_handwriting",
            "UP2_HOBBY":"code_upd2208_doing_hobbies_and_other_activities",
            "UP2_BED":"code_upd2209_turning_in_bed",
            "UP2_TREM":"code_upd2210_tremor",
            "UP2_GETUP":"code_upd2211_get_out_of_bed_car_or_deep_chair",
            "UP2_WALK":"code_upd2212_walking_and_balance",
            "UP2_FREEZ":"code_upd2213_freezing",
            "UP2":'mds_updrs_part_ii_summary_score',
            "UP3":'mds_updrs_part_iii_summary_score',
            "APAT":"code_upd2105_apathy",
            "COG":'moca_total_score',
}


def PrepKMSTED(data, cohort_dict):
  new_data = pd.DataFrame()

  flag = 0
  data.loc[:,"ID"] = data.iloc[:,0]
  data = data.loc[:,data.columns.isin(["ID"] + list(cohort_dict.values()))]
  data = data.groupby("ID").fillna(method="bfill")
  for key, val in cohort_dict.items():
    if "IS_" in key:
      new_data.loc[:,key] = [np.nan if pd.isnull(i) else 1 if i >= int(val.split("/")[1]) else 0 for i in data[val.split("/")[0]]]
    elif val not in data.columns:
      print(f"Error: {val} not found")
    else:
      new_data.loc[:,key] = data[val]
      if flag == 1:
        new_data.loc[~new_data.ID.duplicated(keep="first"),f"BL_{key}"] = new_data.loc[:,key]
        new_data.loc[:,f"BL_{key}"] = new_data.loc[:,f"BL_{key}"].fillna(method = "ffill")
      if key == "BL_AGE":
        flag = 1

  TD_data = new_data.copy()
  TD_data.loc[:,"LTF_TD"] = 0 #[0 if k == 0 else 1 if i > j else 0 for i, j, k in zip(TD_data.DATE, TD_data.CENS_DT, TD_data.LTF)]
  TD_data.loc[(~TD_data.duplicated(subset = "ID",keep="last")) & (TD_data.LTF == 1), "LTF_TD"] = 1

  TD_data.loc[:,"TSTART"] = TD_data.groupby("ID").shift(1).loc[:,"daysB"]
  TD_data = TD_data[TD_data.TSTART.notna()]
  new_data = new_data.drop_duplicates(subset="ID",keep="first")
  return(new_data, TD_data)


STED_CS, STED_LNG = PrepKMSTED(STED, STED_cols)

# PPMI_CRA = PPMI_CS.copy()
# PPMI_CRA.loc[:,"CRA_MCI"] = [1 if mca < 26 else 2 if ltf == 1 else 0 for mca,ltf in zip(PPMI_CS.COG,PPMI_CS.LTF)]
# PPMI_CRA.loc[:,"CRA_DEM"] = [1 if mca < 18 else 2 if ltf == 1 else 0 for mca,ltf in zip(PPMI_CS.COG,PPMI_CS.LTF)]
# PPMI_CRA.loc[:,"CRA_DEP"] = [1 if dep == 1 else 2 if ltf == 1 else 0 for dep,ltf in zip(PPMI_CS.DEP,PPMI_CS.LTF)]

STED_CS.to_csv('STED_KM_CS.csv', index=False)
STED_LNG.to_csv('STED_KM_LNG.csv', index=False)
STED_CS

# Tables & Figures

## PPMI

In [ ]:
import tableone as tab1

pdat = PPMI_CS[PPMI_CS.STUDY_ARM == "PD"]

pdat = pdat.merge(PPMI_LNG.drop_duplicates(subset = "ID", keep = "last").loc[:,["ID","VM"]].rename({"VM":"FUTIME"},axis=1), how = "left")
# pdat = pdat[pdat.FUTIME > 0]
pdat.loc[:,"FUTIME"] = (pdat.FUTIME / 12).round(1)
# pdat = pdat[pdat.PHENO == "PD"]
pdat.loc[:,"IS_COG"] = ["DEM" if c < 22 else "MCI" if c < 26 else "NORM" for c in pdat.COG]
# pdat.loc[:,"EDUC"] = ["College" if e >= 16 else "No College" for e in pdat.EDUC]

pdat
tab1.TableOne(pdat.dropna(), columns = ["BL_AGE","durdx","FUTIME", "RACE", "SEX", "EDUC", "UP2", "UP3", "IS_COG", "IS_DEP"],
              nonnormal = ["durdx","FUTIME","UP2","UP3"], groupby = ["LTF"])

In [ ]:
import tableone as tab1

pdat = PPMI_CS[PPMI_CS.STUDY_ARM == "PD"]

pdat = pdat.merge(PPMI_LNG.drop_duplicates(subset = "ID", keep = "last").loc[:,["ID","VM"]].rename({"VM":"FUTIME"},axis=1), how = "left")
# pdat = pdat[pdat.FUTIME > 0]
pdat.loc[:,"FUTIME"] = (pdat.FUTIME / 12).round(1)
# pdat = pdat[pdat.PHENO == "PD"]
pdat.loc[:,"IS_COG"] = ["DEM" if c < 22 else "MCI" if c < 26 else "NORM" for c in pdat.COG]
# pdat.loc[:,"EDUC"] = ["College" if e >= 16 else "No College" for e in pdat.EDUC]

pdat
tab1.TableOne(pdat.dropna(), columns = ["BL_AGE","durdx","FUTIME", "RACE", "SEX", "EDUC", "UP2", "UP3", "IS_COG", "IS_DEP"],
              nonnormal = ["durdx","FUTIME","UP2","UP3"], groupby = ["LTF"])

## NET-PD-LS1

In [ ]:
pdat

In [ ]:
import tableone as tab1

pdat = LS1_CS.copy()

pdat = pdat.merge(LS1_LNG.drop_duplicates(subset = "ID", keep = "last").loc[:,["ID","VM"]].rename({"VM":"FUTIME"},axis=1), how = "left")
# pdat = pdat[pdat.FUTIME > 0]
pdat.loc[:,"FUTIME"] = (pdat.FUTIME / 12).round(1)
# pdat = pdat[pdat.PHENO == "PD"]
pdat.loc[:,"IS_COG"] = ["DEM" if c < 22 else "MCI" if c < 26 else "NORM" for c in pdat.COG]
pdat.loc[:,"EDUC"] = ["College" if e == 1 else "No College" for e in pdat.EDUC]

pdat
tab1.TableOne(pdat, columns = ["BL_AGE","durdx","FUTIME", "RACE", "SEX", "EDUC", "UP2", "UP3", "IS_COG", "IS_DEP"],
              nonnormal = ["durdx","FUTIME","UP2","UP3"], groupby = ["LTF"])

# Archive

In [ ]:
# APOE = pd.read_csv(f"{datafolder}/PPMI.APOE.raw",sep="\t")

# APOE.loc[:,"participant_id"] = [int(i.split("-")[-1]) for i in APOE.IID]

# APOE = APOE.iloc[:,[-1,-2,-3]]
# APOE.columns = ["participant_id","rs429358","rs7412"]

# #Convert to number of risk alleles
# APOE.loc[:,"rs429358"] = [abs(n-2) for n in APOE.rs429358]
# APOE.loc[:,"rs7412"] = [abs(n-2) for n in APOE.rs7412]

# ktdt = ktd.copy()

# ktdt.loc[:,"IS_MCI"] = [1 if m < 26 else 0 for m in ktdt.MOCA]
# ktdt.loc[:,"IS_DEM"] = [1 if m < 21 else 0 for m in ktdt.MOCA]

# for col in ["IS_MCI","IS_DEM"]:
#   temp = APOE.merge(rmTrail(ktdt, "IS_MCI"))
#   temp.to_csv(f"k{col}.csv")

# APOE

In [ ]:
# %%R
# # install.packages("cmprsk")
# # download.file("http://www.stat.unipg.it/luca/misc/CumIncidence.R", destfile = "CumIncidence.R")
# source("CumIncidence.R")

# PPMI_CRA <- fread("PPMI_CRA.csv") %>%
#             filter(PHENO %in% c("PD","Control"))

# fit = CumIncidence(PPMI_CRA$daysB, PPMI_CRA$CRA_DEP, PPMI_CRA$PHENO, cencode = 0, xlab="Days Baseline")


# CI.overall <- cuminc(ftime = PPMI_CRA$daysB, fstatus = PPMI_CRA$CRA_DEP)
# plot(CI.overall, curvlab = c("Attrition", "Depression"), xlab = "Days")

# CI.overall <- cuminc(ftime = PPMI_CRA$daysB, fstatus = PPMI_CRA$CRA_MCI)
# plot(CI.overall, curvlab = c("Attrition", "MCI"), xlab = "Days")

# CI.overall <- cuminc(ftime = PPMI_CRA$daysB, fstatus = PPMI_CRA$CRA_DEM)
# plot(CI.overall, curvlab = c("Attrition", "Dementia"), xlab = "Days")

# import tableone as tab1

# pdat = PPMI_LNG.drop_duplicates(keep="first", subset = "ID")
# pdat = pdat.merge(PPMI_LNG.drop_duplicates(subset = "ID", keep = "last").loc[:,["ID","VM"]].rename({"VM":"FUTIME"},axis=1), how = "left")
# pdat = pdat[pdat.FUTIME > 0]
# pdat.loc[:,"FUTIME"] = (pdat.FUTIME / 12).round(1)
# pdat = pdat[pdat.PHENO == "PD"]
# pdat.loc[:,"IS_COG"] = ["DEM" if c < 22 else "MCI" if c < 26 else "NORM" for c in pdat.COG]
# pdat.loc[:,"EDUC"] = ["College" if e >= 16 else "No College" for e in pdat.EDUC]

# tab1.TableOne(pdat, columns = ["BL_AGE","durdx","FUTIME", "RACE", "SEX", "EDUC", "UP2", "UP3", "IS_COG", "IS_DEP"],
#               nonnormal = ["durdx","FUTIME","UP2","UP3"], groupby = ["LTF"])